# Piano Plyaing with a robotic hand


### Import relevant libraries.

This project utilizes numpy, pandas, pydrake & pydrake-manipulation, and pygame libraries.

In [1]:
# Imports
import numpy as np
import pydot
import pydrake
from IPython.display import display, HTML, SVG, clear_output

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MeshcatVisualizerCpp, MultibodyPlant, MultibodyPositionToGeometryPose, Parser,
    PiecewisePose, PiecewisePolynomial, Quaternion, RigidTransform, TrajectorySource,
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource,UniformlyRandomRotationMatrix,
    RandomGenerator,InverseKinematics, MeshcatVisualizerParams,Solve,PrismaticJoint,RevoluteJoint,FixedOffsetFrame,
    PiecewisePolynomial, TrajectorySource,PiecewiseQuaternionSlerp,JacobianWrtVariable,MultibodyPositionToGeometryPose,
    AbstractValue, ContactResults,LogVectorOutput
)

from pydrake.multibody.tree import (
    DoorHingeConfig, DoorHinge, LinearSpringDamper, RevoluteJoint, SpatialInertia, PlanarJoint
)

import pandas as pd

import pygame as pg 
import time 

from manipulation import running_as_notebook
from manipulation.scenarios import AddMultibodyTriad, AddShape
from manipulation.meshcat_cpp_utils import (StartMeshcat, MeshcatJointSliders,MeshcatPoseSliders, AddMeshcatTriad)

import meshcat as m
from manipulation.meshcat_utils import plot_mathematical_program
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


#### Start Meshcat Visualizer

In [2]:
# Start the visualizer.

proc, zmq_url, web_url = start_zmq_server_as_subprocess(
    server_args=["--ngrok_http_tunnel"])
    
meshcat = StartMeshcat()
def dataframe(trajectory, times, names):
  assert trajectory.rows() == len(names)
  values = trajectory.vector_values(times)
  data = {'t': times }
  for i in range(len(names)):
    data[names[i]] = values[i,:]
  return pd.DataFrame(data)
  

#### Generate Music Related Libraries and Scores

In [3]:
def generate_chord_library():
    chord_list = []
    chord_list.append("none")
    chord_list.append("C")
    chord_list.append("C#")
    chord_list.append("D")
    chord_list.append("D#")
    chord_list.append("E")
    chord_list.append("F")
    chord_list.append("F#")
    chord_list.append("G")
    chord_list.append("G#")
    chord_list.append("A")
    chord_list.append("A#")
    chord_list.append("B")

    chord_list.append("CM")
    chord_list.append("C#M")
    chord_list.append("DM")
    chord_list.append("D#M")
    chord_list.append("EM")
    chord_list.append("FM")
    chord_list.append("F#M")
    chord_list.append("GM")
    chord_list.append("G#M")
    chord_list.append("AM")
    chord_list.append("A#M")
    chord_list.append("BM")

    chord_list.append("Cm")
    chord_list.append("C#m")
    chord_list.append("Dm")
    chord_list.append("D#m")
    chord_list.append("Em")
    chord_list.append("Fm")
    chord_list.append("F#m")
    chord_list.append("Gm")
    chord_list.append("G#m")
    chord_list.append("Am")
    chord_list.append("A#m")
    chord_list.append("Bm")

    chord_dictionary = {}
    chord_dictionary["none"] =["none","none","none","none"]
    chord_dictionary["C"] = ["C"]
    chord_dictionary["C#"] = ["C#"]
    chord_dictionary["D"] = ["D"]
    chord_dictionary["D#"] = ["D#"]
    chord_dictionary["E"] = ["E"]
    chord_dictionary["F"] =["F"]
    chord_dictionary["F#"] = ["F#"]
    chord_dictionary["G"] = ["G"]
    chord_dictionary["G#"] = ["G#"]
    chord_dictionary["A"] = ["A"]
    chord_dictionary["A#"] = ["A#"]
    chord_dictionary["B"] = ["B"]

    chord_dictionary["CM"] = ["C","E","G"]
    chord_dictionary["C#M"] = ["C#","F","G#"]
    chord_dictionary["DM"] = ["D","F#","A"]
    chord_dictionary["D#M"] = ["D#","G","A#"]
    chord_dictionary["EM"] = ["E","G#","B"]
    chord_dictionary["FM"] =["F","A","C"]
    chord_dictionary["F#M"] = ["F#","A#","C#"]
    chord_dictionary["GM"] = ["G","B","D"]
    chord_dictionary["G#M"] = ["G#","C","D#"]
    chord_dictionary["AM"] = ["A","C#","E"]
    chord_dictionary["A#M"] = ["A#","D","F"]
    chord_dictionary["BM"] = ["B","D#","F#"]

    chord_dictionary["Cm"] = ["C","D#","G"]
    chord_dictionary["C#m"] = ["C#","E","G#"]
    chord_dictionary["Dm"] = ["D","F","A"]
    chord_dictionary["D#m"] = ["D#","F#","A#"]
    chord_dictionary["Em"] = ["E","G","B"]
    chord_dictionary["Fm"] =["F","G#","C"]
    chord_dictionary["F#m"] = ["F#","A","C#"]
    chord_dictionary["Gm"] = ["G","A#","D"]
    chord_dictionary["G#m"] = ["G#","B","D#"]
    chord_dictionary["Am"] = ["A","C","E"]
    chord_dictionary["A#m"] = ["A#","C#","F"]
    chord_dictionary["Bm"] = ["B","D","F#"]

    chord_scale_index = {}
    chord_scale_index["none"] =[0,0,0,0]
    chord_scale_index["C"] = [0]
    chord_scale_index["C#"] = [0]
    chord_scale_index["D"] = [0]
    chord_scale_index["D#"] = [0]
    chord_scale_index["E"] = [0]
    chord_scale_index["F"] =[0]
    chord_scale_index["F#"] = [0]
    chord_scale_index["G"] = [0]
    chord_scale_index["G#"] = [0]
    chord_scale_index["A"] = [0]
    chord_scale_index["A#"] = [0]
    chord_scale_index["B"] = [0]

    chord_scale_index["CM"] = [0,0,0]
    chord_scale_index["C#M"] = [0,0,0]
    chord_scale_index["DM"] = [0,0,0]
    chord_scale_index["D#M"] = [0,0,0]
    chord_scale_index["EM"] = [0,0,0]
    chord_scale_index["FM"] =[0,0,1]
    chord_scale_index["F#M"] = [0,0,1]
    chord_scale_index["GM"] = [0,0,1]
    chord_scale_index["G#M"] = [0,1,1]
    chord_scale_index["AM"] = [0,1,1]
    chord_scale_index["A#M"] =[0,1,1]
    chord_scale_index["BM"] = [0,1,1]

    chord_scale_index["Cm"] = [0,0,0]
    chord_scale_index["C#m"] = [0,0,0]
    chord_scale_index["Dm"] = [0,0,0]
    chord_scale_index["D#m"] = [0,0,0]
    chord_scale_index["Em"] = [0,0,0]
    chord_scale_index["Fm"] = [0,0,1]
    chord_scale_index["F#m"] = [0,0,1]
    chord_scale_index["Gm"] = [0,0,1]
    chord_scale_index["G#m"] = [0,0,1]
    chord_scale_index["Am"] = [0,1,1]
    chord_scale_index["A#m"] = [0,1,1]
    chord_scale_index["Bm"] = [0,1,1]
    # Append Dominant7, Major7, Minor7 Later

    return chord_list, chord_dictionary, chord_scale_index

def add_chord_library_song_salute_de_amure(chord_list, chord_dictionary, chord_scale_index):
    chord_list.append("G#BE")
    chord_list.append("F#C#E")
    chord_list.append("F#BD#")
    chord_list.append("BD#")
    chord_list.append("G#C")
    chord_list.append("C#E")
    chord_list.append("F#BE")
    chord_list.append("F#A#E")
    chord_list.append("G#B")
    chord_list.append("C#F")
    chord_list.append("C#A")
    chord_list.append("C#G#")
    chord_list.append("C#F#")
    chord_list.append("AC#")
    chord_list.append("F#B")
    chord_list.append("AB")


    chord_list.append("C#C#")
    chord_list.append("BB")
    chord_list.append("AA")
    chord_list.append("G#EG#")
    chord_list.append("EG#")
    chord_list.append("D#C#F#")
    chord_list.append("C#F#")

    chord_list.append("AE")
    chord_list.append("EG#E")
    chord_list.append("G#E")


    chord_dictionary["G#BE"] = ["G#","B","E"]
    chord_dictionary["F#C#E"] = ["F#","C#","E"]
    chord_dictionary["F#BD#"] = ["F#","B","D#"]
    chord_dictionary["BD#"] = ["B","D#"]
    chord_dictionary["G#C"] = ["G#","C"]
    chord_dictionary["C#E"] = ["C#","E"]
    chord_dictionary["F#BE"] = ["F#","B","E"]
    chord_dictionary["F#A#E"] = ["F#","A#","E"]
    chord_dictionary["G#B"] = ["G#","B"]
    chord_dictionary["C#F"] = ["C#","F"]
    chord_dictionary["C#A"] = ["C#","A"]
    chord_dictionary["C#G#"] = ["C#","G#"]
    chord_dictionary["C#F#"] = ["C#","F#"]
    chord_dictionary["AC#"] = ["A","C#"]
    chord_dictionary["F#B"] = ["F#","B"]
    chord_dictionary["AB"] = ["A","B"]

    chord_dictionary["C#C#"] = ["C#","C#"]
    chord_dictionary["BB"] = ["B","B"]
    chord_dictionary["AA"] = ["A","A"]
    chord_dictionary["G#EG#"] = ["G#","E","G#"]
    chord_dictionary["EG#"] = ["E","G#"]
    chord_dictionary["D#C#F#"] = ["D#","C#","F#"]
    chord_dictionary["C#F#"] = ["C#","F#"]
    chord_dictionary["AE"] = ["A","E"]
    chord_dictionary["EG#E"] = ["E","G#","E"]
    chord_dictionary["G#E"] = ["G#","E"]

    chord_scale_index["G#BE"] = [0,0,1]
    chord_scale_index["F#C#E"] = [0,1,1]
    chord_scale_index["F#BD#"] = [0,0,1]
    chord_scale_index["BD#"] = [0,1]
    chord_scale_index["G#C"] = [0,1]
    chord_scale_index["C#E"] = [0,0]
    chord_scale_index["F#BE"] = [0,0,1]
    chord_scale_index["F#A#E"] = [0,0,1]
    chord_scale_index["G#B"] = [0,0]
    chord_scale_index["C#F"] = [0,0]
    chord_scale_index["C#A"] = [0,0]
    chord_scale_index["C#G#"] = [0,0]
    chord_scale_index["C#F#"] = [0,0]
    chord_scale_index["AC#"] = [0,1]
    chord_scale_index["F#B"] = [0,0]
    chord_scale_index["AB"] = [0,0]


    chord_scale_index["C#C#"] = [0,1]
    chord_scale_index["BB"] = [0,1]
    chord_scale_index["AA"] = [0,1]
    chord_scale_index["G#EG#"] = [0,1,1]
    chord_scale_index["EG#"] = [0,0]
    chord_scale_index["D#C#F#"] = [0,1,1]
    chord_scale_index["C#F#"] = [0,0]
    chord_scale_index["AE"] = [0,1]
    chord_scale_index["EG#E"] = [0,0,1]
    chord_scale_index["G#E"] = [0,1]


    
    return chord_list, chord_dictionary, chord_scale_index

chord_list, chord_dictionary, chord_scale_index = generate_chord_library()
chord_list, chord_dictionary, chord_scale_index = add_chord_library_song_salute_de_amure(chord_list, chord_dictionary, chord_scale_index)
# print(len(chord_list))

def generate_note_from_chord(chord_name, scale=0):
    notes = chord_dictionary[chord_name]
    scale_offset = chord_scale_index[chord_name]

    C_out = []
    for idx, note in enumerate(notes):
        s = scale+scale_offset[idx]
        C_out.append(note+"_{}".format(s))
    return C_out


class Music_Note():
    def __init__(self, name, chord=None, key_scale = 0, t_push=0.16, t_hold = 0.4, t_release = 0.12,t_transition=0.32,t_scale=1):
        self.name = name
        if chord ==None:
            self.chord = chord_dictionary[name]
        else:
            self.chord = chord

        my_scaler = 1
        self.key_scale = key_scale
        self.t_push = t_push*my_scaler*t_scale
        self.t_release = t_release *my_scaler*t_scale
        self.t_transition = t_transition*my_scaler*t_scale
        self.t_hold = (t_hold+t_push+t_release+t_transition)*t_scale*my_scaler - self.t_push - self.t_release -self.t_transition

def generate_little_star_chords_ver():
    note_seq = []
    note_seq.append(Music_Note("CM"))
    note_seq.append(Music_Note("CM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("AM"))
    note_seq.append(Music_Note("AM"))
    note_seq.append(Music_Note("GM",t_scale = 2))

    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("DM"))
    note_seq.append(Music_Note("DM"))
    note_seq.append(Music_Note("CM",t_scale = 2))

    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("DM",t_scale = 2))

    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("DM",t_scale = 2))

    note_seq.append(Music_Note("CM"))
    note_seq.append(Music_Note("CM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("AM"))
    note_seq.append(Music_Note("AM"))
    note_seq.append(Music_Note("GM",t_scale = 2))

    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("DM"))
    note_seq.append(Music_Note("DM"))
    note_seq.append(Music_Note("CM",t_scale = 2))

    return note_seq
    

def generate_little_star_note_ver():
    note_seq = []
    note_seq.append(Music_Note("C"))
    note_seq.append(Music_Note("C"))
    note_seq.append(Music_Note("G"))
    note_seq.append(Music_Note("G"))
    note_seq.append(Music_Note("A"))
    note_seq.append(Music_Note("A"))
    note_seq.append(Music_Note("G",t_scale = 2))

    note_seq.append(Music_Note("F"))
    note_seq.append(Music_Note("F"))
    note_seq.append(Music_Note("E"))
    note_seq.append(Music_Note("E"))
    note_seq.append(Music_Note("D"))
    note_seq.append(Music_Note("D"))
    note_seq.append(Music_Note("C",t_scale = 2))

    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("D",key_scale =1,t_scale = 2))

    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("D",key_scale =1,t_scale = 2))

    note_seq.append(Music_Note("C",key_scale =1))
    note_seq.append(Music_Note("C",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("A",key_scale =1))
    note_seq.append(Music_Note("A",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1,t_scale = 2))

    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("D",key_scale =1))
    note_seq.append(Music_Note("D",key_scale =1))
    note_seq.append(Music_Note("C",key_scale =1,t_scale = 2))

    return note_seq





def generate_chord_library():
    chord_list = []
    chord_list.append("none")
    chord_list.append("C")
    chord_list.append("C#")
    chord_list.append("D")
    chord_list.append("D#")
    chord_list.append("E")
    chord_list.append("F")
    chord_list.append("F#")
    chord_list.append("G")
    chord_list.append("G#")
    chord_list.append("A")
    chord_list.append("A#")
    chord_list.append("B")

    chord_list.append("CM")
    chord_list.append("C#M")
    chord_list.append("DM")
    chord_list.append("D#M")
    chord_list.append("EM")
    chord_list.append("FM")
    chord_list.append("F#M")
    chord_list.append("GM")
    chord_list.append("G#M")
    chord_list.append("AM")
    chord_list.append("A#M")
    chord_list.append("BM")

    chord_list.append("Cm")
    chord_list.append("C#m")
    chord_list.append("Dm")
    chord_list.append("D#m")
    chord_list.append("Em")
    chord_list.append("Fm")
    chord_list.append("F#m")
    chord_list.append("Gm")
    chord_list.append("G#m")
    chord_list.append("Am")
    chord_list.append("A#m")
    chord_list.append("Bm")

    chord_dictionary = {}
    chord_dictionary["none"] =["none","none","none","none"]
    chord_dictionary["C"] = ["C"]
    chord_dictionary["C#"] = ["C#"]
    chord_dictionary["D"] = ["D"]
    chord_dictionary["D#"] = ["D#"]
    chord_dictionary["E"] = ["E"]
    chord_dictionary["F"] =["F"]
    chord_dictionary["F#"] = ["F#"]
    chord_dictionary["G"] = ["G"]
    chord_dictionary["G#"] = ["G#"]
    chord_dictionary["A"] = ["A"]
    chord_dictionary["A#"] = ["A#"]
    chord_dictionary["B"] = ["B"]

    chord_dictionary["CM"] = ["C","E","G"]
    chord_dictionary["C#M"] = ["C#","F","G#"]
    chord_dictionary["DM"] = ["D","F#","A"]
    chord_dictionary["D#M"] = ["D#","G","A#"]
    chord_dictionary["EM"] = ["E","G#","B"]
    chord_dictionary["FM"] =["F","A","C"]
    chord_dictionary["F#M"] = ["F#","A#","C#"]
    chord_dictionary["GM"] = ["G","B","D"]
    chord_dictionary["G#M"] = ["G#","C","D#"]
    chord_dictionary["AM"] = ["A","C#","E"]
    chord_dictionary["A#M"] = ["A#","D","F"]
    chord_dictionary["BM"] = ["B","D#","F#"]

    chord_dictionary["Cm"] = ["C","D#","G"]
    chord_dictionary["C#m"] = ["C#","E","G#"]
    chord_dictionary["Dm"] = ["D","F","A"]
    chord_dictionary["D#m"] = ["D#","F#","A#"]
    chord_dictionary["Em"] = ["E","G","B"]
    chord_dictionary["Fm"] =["F","G#","C"]
    chord_dictionary["F#m"] = ["F#","A","C#"]
    chord_dictionary["Gm"] = ["G","A#","D"]
    chord_dictionary["G#m"] = ["G#","B","D#"]
    chord_dictionary["Am"] = ["A","C","E"]
    chord_dictionary["A#m"] = ["A#","C#","F"]
    chord_dictionary["Bm"] = ["B","D","F#"]

    chord_scale_index = {}
    chord_scale_index["none"] =[0,0,0,0]
    chord_scale_index["C"] = [0]
    chord_scale_index["C#"] = [0]
    chord_scale_index["D"] = [0]
    chord_scale_index["D#"] = [0]
    chord_scale_index["E"] = [0]
    chord_scale_index["F"] =[0]
    chord_scale_index["F#"] = [0]
    chord_scale_index["G"] = [0]
    chord_scale_index["G#"] = [0]
    chord_scale_index["A"] = [0]
    chord_scale_index["A#"] = [0]
    chord_scale_index["B"] = [0]

    chord_scale_index["CM"] = [0,0,0]
    chord_scale_index["C#M"] = [0,0,0]
    chord_scale_index["DM"] = [0,0,0]
    chord_scale_index["D#M"] = [0,0,0]
    chord_scale_index["EM"] = [0,0,0]
    chord_scale_index["FM"] =[0,0,1]
    chord_scale_index["F#M"] = [0,0,1]
    chord_scale_index["GM"] = [0,0,1]
    chord_scale_index["G#M"] = [0,1,1]
    chord_scale_index["AM"] = [0,1,1]
    chord_scale_index["A#M"] =[0,1,1]
    chord_scale_index["BM"] = [0,1,1]

    chord_scale_index["Cm"] = [0,0,0]
    chord_scale_index["C#m"] = [0,0,0]
    chord_scale_index["Dm"] = [0,0,0]
    chord_scale_index["D#m"] = [0,0,0]
    chord_scale_index["Em"] = [0,0,0]
    chord_scale_index["Fm"] = [0,0,1]
    chord_scale_index["F#m"] = [0,0,1]
    chord_scale_index["Gm"] = [0,0,1]
    chord_scale_index["G#m"] = [0,0,1]
    chord_scale_index["Am"] = [0,1,1]
    chord_scale_index["A#m"] = [0,1,1]
    chord_scale_index["Bm"] = [0,1,1]
    # Append Dominant7, Major7, Minor7 Later

    return chord_list, chord_dictionary, chord_scale_index

def add_chord_library_song_salute_de_amure(chord_list, chord_dictionary, chord_scale_index):
    chord_list.append("G#BE")
    chord_list.append("F#C#E")
    chord_list.append("F#BD#")
    chord_list.append("BD#")
    chord_list.append("G#C")
    chord_list.append("C#E")
    chord_list.append("F#BE")
    chord_list.append("F#A#E")
    chord_list.append("G#B")
    chord_list.append("C#F")
    chord_list.append("C#A")
    chord_list.append("C#G#")
    chord_list.append("C#F#")
    chord_list.append("AC#")
    chord_list.append("F#B")
    chord_list.append("AB")


    chord_list.append("C#C#")
    chord_list.append("BB")
    chord_list.append("AA")
    chord_list.append("G#EG#")
    chord_list.append("EG#")
    chord_list.append("D#C#F#")
    chord_list.append("C#F#")

    chord_list.append("AE")
    chord_list.append("EG#E")
    chord_list.append("G#E")


    chord_dictionary["G#BE"] = ["G#","B","E"]
    chord_dictionary["F#C#E"] = ["F#","C#","E"]
    chord_dictionary["F#BD#"] = ["F#","B","D#"]
    chord_dictionary["BD#"] = ["B","D#"]
    chord_dictionary["G#C"] = ["G#","C"]
    chord_dictionary["C#E"] = ["C#","E"]
    chord_dictionary["F#BE"] = ["F#","B","E"]
    chord_dictionary["F#A#E"] = ["F#","A#","E"]
    chord_dictionary["G#B"] = ["G#","B"]
    chord_dictionary["C#F"] = ["C#","F"]
    chord_dictionary["C#A"] = ["C#","A"]
    chord_dictionary["C#G#"] = ["C#","G#"]
    chord_dictionary["C#F#"] = ["C#","F#"]
    chord_dictionary["AC#"] = ["A","C#"]
    chord_dictionary["F#B"] = ["F#","B"]
    chord_dictionary["AB"] = ["A","B"]

    chord_dictionary["C#C#"] = ["C#","C#"]
    chord_dictionary["BB"] = ["B","B"]
    chord_dictionary["AA"] = ["A","A"]
    chord_dictionary["G#EG#"] = ["G#","E","G#"]
    chord_dictionary["EG#"] = ["E","G#"]
    chord_dictionary["D#C#F#"] = ["D#","C#","F#"]
    chord_dictionary["C#F#"] = ["C#","F#"]
    chord_dictionary["AE"] = ["A","E"]
    chord_dictionary["EG#E"] = ["E","G#","E"]
    chord_dictionary["G#E"] = ["G#","E"]

    chord_scale_index["G#BE"] = [0,0,1]
    chord_scale_index["F#C#E"] = [0,1,1]
    chord_scale_index["F#BD#"] = [0,0,1]
    chord_scale_index["BD#"] = [0,1]
    chord_scale_index["G#C"] = [0,1]
    chord_scale_index["C#E"] = [0,0]
    chord_scale_index["F#BE"] = [0,0,1]
    chord_scale_index["F#A#E"] = [0,0,1]
    chord_scale_index["G#B"] = [0,0]
    chord_scale_index["C#F"] = [0,0]
    chord_scale_index["C#A"] = [0,0]
    chord_scale_index["C#G#"] = [0,0]
    chord_scale_index["C#F#"] = [0,0]
    chord_scale_index["AC#"] = [0,1]
    chord_scale_index["F#B"] = [0,0]
    chord_scale_index["AB"] = [0,0]


    chord_scale_index["C#C#"] = [0,1]
    chord_scale_index["BB"] = [0,1]
    chord_scale_index["AA"] = [0,1]
    chord_scale_index["G#EG#"] = [0,1,1]
    chord_scale_index["EG#"] = [0,0]
    chord_scale_index["D#C#F#"] = [0,1,1]
    chord_scale_index["C#F#"] = [0,0]
    chord_scale_index["AE"] = [0,1]
    chord_scale_index["EG#E"] = [0,0,1]
    chord_scale_index["G#E"] = [0,1]



    
    return chord_list, chord_dictionary, chord_scale_index

chord_list, chord_dictionary, chord_scale_index = generate_chord_library()
chord_list, chord_dictionary, chord_scale_index = add_chord_library_song_salute_de_amure(chord_list, chord_dictionary, chord_scale_index)
# print(len(chord_list))

def generate_note_from_chord(chord_name, scale=0):
    notes = chord_dictionary[chord_name]
    scale_offset = chord_scale_index[chord_name]

    C_out = []
    for idx, note in enumerate(notes):
        s = scale+scale_offset[idx]
        C_out.append(note+"_{}".format(s))
    return C_out


class Music_Note():
    def __init__(self, name, chord=None, key_scale = 0, t_push=0.16, t_hold = 0.4, t_release = 0.12,t_transition=0.32,t_scale=1):
        self.name = name
        if chord ==None:
            self.chord = chord_dictionary[name]
        else:
            self.chord = chord

        my_scaler = 1
        self.key_scale = key_scale
        self.t_push = t_push*my_scaler*t_scale
        self.t_release = t_release *my_scaler*t_scale
        self.t_transition = t_transition*my_scaler*t_scale
        self.t_hold = (t_hold+t_push+t_release+t_transition)*t_scale*my_scaler - self.t_push - self.t_release -self.t_transition

def generate_little_star_chords_ver():
    note_seq = []
    note_seq.append(Music_Note("CM"))
    note_seq.append(Music_Note("CM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("AM"))
    note_seq.append(Music_Note("AM"))
    note_seq.append(Music_Note("GM",t_scale = 2))

    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("DM"))
    note_seq.append(Music_Note("DM"))
    note_seq.append(Music_Note("CM",t_scale = 2))

    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("DM",t_scale = 2))

    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("DM",t_scale = 2))

    note_seq.append(Music_Note("CM"))
    note_seq.append(Music_Note("CM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("GM"))
    note_seq.append(Music_Note("AM"))
    note_seq.append(Music_Note("AM"))
    note_seq.append(Music_Note("GM",t_scale = 2))

    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("FM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("EM"))
    note_seq.append(Music_Note("DM"))
    note_seq.append(Music_Note("DM"))
    note_seq.append(Music_Note("CM",t_scale = 2))

    return note_seq
    

def generate_little_star_note_ver():
    note_seq = []
    note_seq.append(Music_Note("C"))
    note_seq.append(Music_Note("C"))
    note_seq.append(Music_Note("G"))
    note_seq.append(Music_Note("G"))
    note_seq.append(Music_Note("A"))
    note_seq.append(Music_Note("A"))
    note_seq.append(Music_Note("G",t_scale = 2))

    note_seq.append(Music_Note("F"))
    note_seq.append(Music_Note("F"))
    note_seq.append(Music_Note("E"))
    note_seq.append(Music_Note("E"))
    note_seq.append(Music_Note("D"))
    note_seq.append(Music_Note("D"))
    note_seq.append(Music_Note("C",t_scale = 2))

    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("D",key_scale =1,t_scale = 2))

    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("D",key_scale =1,t_scale = 2))

    note_seq.append(Music_Note("C",key_scale =1))
    note_seq.append(Music_Note("C",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1))
    note_seq.append(Music_Note("A",key_scale =1))
    note_seq.append(Music_Note("A",key_scale =1))
    note_seq.append(Music_Note("G",key_scale =1,t_scale = 2))

    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("F",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("E",key_scale =1))
    note_seq.append(Music_Note("D",key_scale =1))
    note_seq.append(Music_Note("D",key_scale =1))
    note_seq.append(Music_Note("C",key_scale =1,t_scale = 2))

    return note_seq

        
        


def generate_salute_de_amur_lowchord():
    bpm = 67
    ts = 60/bpm

    note_seq = []


    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))
#---------------------------------------------------------------------------

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))


    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))
#---------------------------------------------------------------------------


    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#C#E",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#C#E",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

#--------------------------------------------------------------------------

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#BD#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#C",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

#--------------------------------------------------------------------------



    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))
#------------------------------------------------------------------------------------
    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#A#E",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))
#----------------------------------------------------------------------------

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    #--------------------------------------------------
    bpm = 60
    ts = 60/bpm

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))
#------------------------------------------------------------------------

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#F",key_scale =1, t_scale = t_note)) 
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

#------------------------------------------------------------------------

    bpm = 50
    ts = 60/bpm

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#A",key_scale =1, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#G#",key_scale =1, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#F#",key_scale =1, t_scale = t_note)) 
#-------------------------------------------------------------------------

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("AC#",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =1, t_scale = t_note)) 
#----------------------------------------------------------------

    bpm = 40
    ts = 60/bpm
    
    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("AB",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))
#-------------------------------------------------------------------

    note_size = 1.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =0, t_scale = t_note)) 


    return note_seq



def generate_salute_de_amur_lowchord_offbeat():
    bpm = 67
    ts = 60/bpm

    note_seq = []

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#C#E",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#C#E",key_scale =0, t_scale = t_note))
#--------------------------------------------------------------------------
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#BD#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#C",key_scale =0, t_scale = t_note))



    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#A#E",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =0, t_scale = t_note))

    #--------------------------------------------------
    bpm = 60
    ts = 60/bpm

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#F",key_scale =1, t_scale = t_note)) 


    bpm = 50
    ts = 60/bpm

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#A",key_scale =1, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#G#",key_scale =1, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#F#",key_scale =1, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("AC#",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =1, t_scale = t_note)) 
#----------------------------------------------------------------

    bpm = 40
    ts = 60/bpm
    
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("AB",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =0, t_scale = t_note)) 


    return note_seq


def generate_salute_de_amur_lowchord_5actave():
    bpm = 67
    ts = 60/bpm

    note_seq = []


    note_size = 1.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =1, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =1, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))
#---------------------------------------------------------
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#C#E",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#C#E",key_scale =1, t_scale = t_note))
#--------------------------------------------------------------------------
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#BD#",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =1, t_scale = t_note))
#--------------------------------------------------------------------------

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =1, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#C",key_scale =1, t_scale = t_note))



    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#",key_scale =1, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =2, t_scale = t_note))
#--------------------------------------------------------------------------

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#A#E",key_scale =1, t_scale = t_note))
#--------------------------------------------------------------------------

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#BD#",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =1, t_scale = t_note))

#--------------------------------------------------------------------------
    bpm = 60
    ts = 60/bpm

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#BE",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#B",key_scale =1, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#B",key_scale =1, t_scale = t_note)) 

#--------------------------------------------------------------------------
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#F",key_scale =2, t_scale = t_note)) 


    bpm = 50
    ts = 60/bpm

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#A",key_scale =2, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#G#",key_scale =2, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#F#",key_scale =2, t_scale = t_note)) 

#--------------------------------------------------------------------------
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =0, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =1, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("AC#",key_scale =1, t_scale = t_note)) 

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#E",key_scale =2, t_scale = t_note)) 
#----------------------------------------------------------------

    bpm = 40
    ts = 60/bpm
    
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =0, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#B",key_scale =1, t_scale = t_note)) 


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =0, t_scale = t_note))
    
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("AB",key_scale =1, t_scale = t_note)) 

#----------------------------------------------------------------
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =0, t_scale = t_note))

    note_size = 1.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =1, t_scale = t_note)) 


    return note_seq


def generate_salute_de_amur_highchord_5actave():
    bpm = 67
    ts = 60/bpm

    note_seq = []

    note_size = 4
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =1, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =3, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =3, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("D#",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =3, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =3, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =3, t_scale = t_note))



#--------------------------------------------------------------------------

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =1, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =3, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C",key_scale =3, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =3, t_scale = t_note))



    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("D#",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =3, t_scale = t_note))



    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =3, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =3, t_scale = t_note))

    note_size = 1.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =3, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G",key_scale =3, t_scale = t_note))

    #--------------------------------------------------
    bpm = 60
    ts = 60/bpm

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =3, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =3, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("D#",key_scale =3, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =3, t_scale = t_note))
    ##

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#",key_scale =4, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#",key_scale =4, t_scale = t_note))


    bpm = 50
    ts = 60/bpm


    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#",key_scale =4, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =3, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =3, t_scale = t_note))



    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("EG#",key_scale =3, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#F#",key_scale =3, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("AE",key_scale =2, t_scale = t_note))


#----------------------------------------------------------------

    bpm = 40
    ts = 60/bpm

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("AC#",key_scale =2, t_scale = t_note)) 

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =2, t_scale = t_note)) 


    note_size = 2
    t_note = note_size*ts
    note_seq.append(Music_Note("G#E",key_scale =2, t_scale = t_note)) 


    return note_seq
    





def generate_salute_de_amur_highchord():
    bpm = 67
    ts = 60/bpm

    note_seq = []

    note_size = 4
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =2, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("D#",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =2, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =2, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =2, t_scale = t_note))



#--------------------------------------------------------------------------

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =1, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("none",key_scale =0, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =2, t_scale = t_note))



    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("D#",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =2, t_scale = t_note))



    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =2, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =2, t_scale = t_note))

    note_size = 1.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G",key_scale =2, t_scale = t_note))

    #--------------------------------------------------
    bpm = 60
    ts = 60/bpm

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =1, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("G#",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("F#",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("D#",key_scale =2, t_scale = t_note))
    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("E",key_scale =2, t_scale = t_note))
    ##

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#",key_scale =3, t_scale = t_note))

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#",key_scale =3, t_scale = t_note))


    bpm = 50
    ts = 60/bpm


    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("C#",key_scale =3, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("B",key_scale =2, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("A",key_scale =2, t_scale = t_note))



    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("EG#",key_scale =2, t_scale = t_note))

    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("C#F#",key_scale =2, t_scale = t_note))


    note_size = 0.5
    t_note = note_size*ts
    note_seq.append(Music_Note("AE",key_scale =1, t_scale = t_note))


#----------------------------------------------------------------

    bpm = 40
    ts = 60/bpm

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("AC#",key_scale =1, t_scale = t_note)) 

    note_size = 1
    t_note = note_size*ts
    note_seq.append(Music_Note("BD#",key_scale =1, t_scale = t_note)) 


    note_size = 2
    t_note = note_size*ts
    note_seq.append(Music_Note("G#E",key_scale =1, t_scale = t_note)) 


    return note_seq





#### Generate Keyboard Key Class

In [9]:
class Keyboard_Key():
    def __init__(self, name="key", pos = np.array([0,0,0]),rotation=RotationMatrix() ,frame = None, pitch = 0,keytype="flat"):
        self.name = name
        self.pos = pos
        self.rotation = rotation
        self.pitch = pitch
        self.frame = frame
        self.keytype = keytype
        


class MyControllerSystem(LeafSystem):
    """Wrapper system for Dynamic torque input
    """ 

    def __init__(self, plant):#, diffik_fun):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        # self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()
        self._iiwa = plant.GetModelInstanceByName("iiwa7")
        self._hand = plant.GetModelInstanceByName("allegro_hand_right")

        finger_name_list = ["link_15", "link_3", "link_7","link_11"]
        self._F0 = plant.GetFrameByName("frame_fingercontact_"+finger_name_list[0])
        self._F1 = plant.GetFrameByName("frame_fingercontact_"+finger_name_list[1])
        self._F2 = plant.GetFrameByName("frame_fingercontact_"+finger_name_list[2])
        self._F3 = plant.GetFrameByName("frame_fingercontact_"+finger_name_list[3])

        self._BF0 = plant.GetBodyByName(finger_name_list[0]).body_frame()
        self._BF1 = plant.GetBodyByName(finger_name_list[1]).body_frame()
        self._BF2 = plant.GetBodyByName(finger_name_list[2]).body_frame()
        self._BF3 = plant.GetBodyByName(finger_name_list[3]).body_frame()
        
        self._P = plant.GetFrameByName("frame_palm")
        
        # define indices of the robot link
        iiwa_indices =np.array([0,61,62,63,64,65,66])
        hand_indices =np.array([67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82])
        self.robot_indices = np.hstack([iiwa_indices,hand_indices] )
        
        # define ports
        self.iiwa7_state_port = self.DeclareVectorInputPort("iiwa7_state", BasicVector(7*2))
        self.hand_state_port = self.DeclareVectorInputPort("hand_state", BasicVector(16*2))

        self.iiwa7_des_pos_port = self.DeclareVectorInputPort("iiwa7_des_pos", BasicVector(7))
        self.hand_des_pos_port = self.DeclareVectorInputPort("hand_des_pos", BasicVector(16))

        self.iiwa7_des_vel_port = self.DeclareVectorInputPort("iiwa7_des_vel", BasicVector(7))
        self.hand_des_vel_port = self.DeclareVectorInputPort("hand_des_vel", BasicVector(16))

        self.finger_des_contact_port = self.DeclareVectorInputPort("finger_des_contact", BasicVector(4))


        # self.hand_contact_force_port = self.DeclareVectorInputPort("hand_contact_forces",16)
        # self.contact_result_port = self.DeclareAbstractInputPort("contact_result", AbstractValue.Make(ContactResults()))


        self.DeclareVectorOutputPort("iiwa7_torque", BasicVector(7),self.calc_iiwa7_torque)
        self.DeclareVectorOutputPort("hand_torque", BasicVector(16),self.calc_hand_torque)
        self.cnt=0
        
        self.u_iiwa =np.zeros((7,1))
        self.u_hand =np.zeros((16,1))
        
    def calc_J(self,q_iiwa,q_hand):
        self._plant.SetPositions(self._plant_context, self._iiwa, q_iiwa)
        self._plant.SetPositions(self._plant_context, self._hand, q_hand)

        J_F0 = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV, 
            self._BF0, [0,0,0], self._W, self._W)
        J_F0 = J_F0[0:3,self.robot_indices]

        J_F1 = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV, 
            self._BF1, [0,0,0], self._W, self._W)
        J_F1 = J_F1[0:3,self.robot_indices]

        J_F2 = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV, 
            self._BF2, [0,0,0], self._W, self._W)
        J_F2 = J_F2[0:3,self.robot_indices]

        J_F3 = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV, 
            self._BF3, [0,0,0], self._W, self._W)
        J_F3 = J_F3[0:3,self.robot_indices]


        return J_F0, J_F1, J_F2, J_F3#, J_P

    def calc_pose(self, q_iiwa,q_hand):
        self._plant.SetPositions(self._plant_context, self._iiwa, q_iiwa)
        self._plant.SetPositions(self._plant_context, self._hand, q_hand)

        pose_F0 = self.get_pose(self._F0)
        pose_F1 = self.get_pose(self._F1)
        pose_F2 = self.get_pose(self._F2)
        pose_F3 = self.get_pose(self._F3)
        #pose_P = self.get_pose(self._P)
        
        return pose_F0, pose_F1, pose_F2, pose_F3#, pose_P

    def get_pose(self, frame):
        return self._plant.CalcRelativeTransform(
                            self._plant_context,
                            frame_A=self._plant.GetFrameByName("frame_top_ground"),
                            frame_B=frame)


    def main_controller(self, context):
        state_iiwa7 =  self.iiwa7_state_port.Eval(context)
        des_pos_iiwa7 =  self.iiwa7_des_pos_port.Eval(context)
        des_vel_iiwa7 =  self.iiwa7_des_vel_port.Eval(context)

        q_iiwa7 =np.zeros((7,1))
        dq_iiwa7 = np.zeros((7,1))

        q_iiwa7[:,0] = state_iiwa7[:7]
        dq_iiwa7[:,0] = state_iiwa7[7:]

        q_iiwa7_des = np.zeros((7,1))
        q_iiwa7_des[:,0] = des_pos_iiwa7
        dq_iiwa7_des = np.zeros((7,1))
        dq_iiwa7_des[:,0] = des_vel_iiwa7
        
        state_hand =  self.hand_state_port.Eval(context)
        des_pos_hand =  self.hand_des_pos_port.Eval(context)
        des_vel_hand =  self.hand_des_vel_port.Eval(context)
        
        q_hand =np.zeros((16,1))
        dq_hand = np.zeros((16,1))

        q_hand[:,0] = state_hand[:16]
        dq_hand[:,0] = state_hand[16:]

        q_hand_des = np.zeros((16,1))
        q_hand_des[:,0] = des_pos_hand
        dq_hand_des = np.zeros((16,1))
        dq_hand_des[:,0] = des_vel_hand

        des_contact_finger = self.finger_des_contact_port.Eval(context)
        
        K_iiwa = 2*np.array([[1000, 800, 800, 700,500,400,300]]).T
        D_iiwa = K_iiwa*0.22
        u_iiwa_pd = np.multiply(K_iiwa,(q_iiwa7_des-q_iiwa7))+np.multiply(D_iiwa,dq_iiwa7_des-dq_iiwa7) 
        self.u_iiwa = u_iiwa_pd
             
        # define baseline joint gains
        K_hand = 1.8*np.array([[2,2,2,2, 1,1,1,1, 1,1,1,1, 0.5,0.5,0.5,0.5]]).T
        
        # apply contact mode gains
        K_hand_contact = K_hand*10
        K_hand_contact[0,0] *= des_contact_finger[1]
        K_hand_contact[4,0] *= des_contact_finger[1]
        K_hand_contact[8,0] *= des_contact_finger[1]
        K_hand_contact[12,0] *= des_contact_finger[1]

        K_hand_contact[1,0] *= des_contact_finger[0]
        K_hand_contact[5,0] *= des_contact_finger[0]
        K_hand_contact[9,0] *= des_contact_finger[0]
        K_hand_contact[13,0] *= des_contact_finger[0]

        K_hand_contact[2,0] *= des_contact_finger[2]
        K_hand_contact[6,0] *= des_contact_finger[2]
        K_hand_contact[10,0] *= des_contact_finger[2]
        K_hand_contact[14,0] *= des_contact_finger[2]

        K_hand_contact[3,0] *= des_contact_finger[3]
        K_hand_contact[7,0] *= des_contact_finger[3]
        K_hand_contact[11,0] *= des_contact_finger[3]
        K_hand_contact[15,0] *= des_contact_finger[3]

        K_hand += K_hand_contact
        
        D_hand = K_hand*0.16
        
        u_hand_pd = np.multiply(K_hand,(q_hand_des-q_hand))+np.multiply(D_hand,dq_hand_des-dq_hand)


        u_hand = u_hand_pd

        # Re-mapping the gains. 
        # This is becasue joint index and actuator index is not aligned in the model.

        self.u_hand = u_hand*0
        self.u_hand[0,0] = u_hand[0,0]
        self.u_hand[4,0] = u_hand[1,0]
        self.u_hand[8,0] = u_hand[2,0]
        self.u_hand[12,0] = u_hand[3,0]

        self.u_hand[1,0] = u_hand[4,0]
        self.u_hand[5,0] = u_hand[5,0]
        self.u_hand[9,0] = u_hand[6,0]
        self.u_hand[13,0] = u_hand[7,0]

        self.u_hand[2,0] = u_hand[8,0]
        self.u_hand[6,0] = u_hand[9,0]
        self.u_hand[10,0] = u_hand[10,0]
        self.u_hand[14,0] = u_hand[11,0]

        self.u_hand[3,0] = u_hand[12,0]
        self.u_hand[7,0] = u_hand[13,0]
        self.u_hand[11,0] = u_hand[14,0]
        self.u_hand[15,0] = u_hand[15,0]

    def calc_iiwa7_torque(self, context, output):
        self.main_controller(context)
 
        output.SetFromVector(self.u_iiwa)

    def calc_hand_torque(self, context, output):
        self.main_controller(context)
        output.SetFromVector(self.u_hand)
            
        
class PianoOutputSystem(LeafSystem):
    """Wrapper system for Piano Sound Output. 
    """ 

    def __init__(self, plant,key_list):#, diffik_fun):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        key_indices = np.arange(1,61)

        self.q_port = self.DeclareVectorInputPort("q", BasicVector(83))
        self.w_port = self.DeclareVectorInputPort("w", BasicVector(83))

        self.DeclareVectorOutputPort("sound_output", BasicVector(60),self.adjust_sound)
        self.DeclareVectorOutputPort("state_output", BasicVector(83),self.data_pipe)
    
        self.key_list = key_list
        self.N_key = len(key_list)

        pg.mixer.init()
        pg.init()
        pg.mixer.set_num_channels(self.N_key)
        self.init_keys(key_list)

        
    class SoundNote:
        def __init__(self, index, name, path):
            self.index = index
            self.name = name
            self.soundpath = path
            self.channel = pg.mixer.Sound(path)
            
            self.t_on = 0
            self.t_off = 0
            self.q_max = 0

            self.volume = 0
            self.play_state = "off"

        def set_volume(self, new_volume=1):
            self.channel.set_volume(new_volume)
            self.volume = new_volume
        
        def play(self, new_volume =1):
            self.channel.play(fade_ms=70)
            self.set_volume(new_volume)
        
        def fadeout(self, t_off = 400):
            self.channel.fadeout(t_off) # fadeout after t_off ms
        
        def stop(self):
            self.channel.stop()

    def init_keys(self,key_list):
        # the sound used in this module 
        # is forked from https://github.com/ledlamp/piano-sounds
        # mp3 files should be reprocessed to wave file
        base_path = "piano-sounds/Steinway_Grand/"

        self.Notes =[]
        n_const =1
        
        for idx, key in enumerate(key_list):            
            key_path = key
            key_path = key_path.replace("_","") #remove underscore
            key_path = key_path.replace("#","s")
            n = int(key_path[-1])+n_const

            key_path = key_path[:-1].lower()
            
            path = base_path+key_path+str(n)+".mp3.wav"
            note = self.SoundNote(idx,key,path)
            self.Notes.append(note)

    def data_pipe(self, context,output):

        q_all =  self.q_port.Eval(context)

        q_state = np.zeros((71+12,1))
        q_state[:,0] = q_all
        self.adjust_sound(context,output)
        output.SetFromVector(q_state)
        
    def adjust_sound(self, context,output):
        time_step = 1e-3
        q_all =  self.q_port.Eval(context)
        w_all =  self.w_port.Eval(context)
        
        q0 = 0.0025
        q_piano = np.zeros((48+12,1))
        w_piano = np.zeros((48+12,1))

        q_piano[:,0] = q_all[1:49+12]-q0
        w_piano[:,0] = w_all[1:49+12]


        q_on_threshold = 0.001 #1degree 
        q_off_threshold = 0.002 #1degree 
        
        q_conv_const = 20
        w_conv_const = 0.3
        def update_note(idx):
            note = self.Notes[idx]
            q_note = q_piano[idx,0]
            w_note = w_piano[idx,0]

            play_state = note.play_state
            
            if play_state =="off":

                if q_note >q_on_threshold:
                    note.play(q_note*q_conv_const)
                    print("idx : {}, name: {}, q: {}, volume: {}".format(note.index, note.name, q_note, q_note*q_conv_const))
                    self.Notes[idx].q_max = q_note
                    play_state = "on_rising"
            
            elif play_state == "on_rising":
                self.Notes[idx].t_on+=time_step

                if self.Notes[idx].q_max < q_note+1e-5:
                    # note.set_volume(q_note*q_conv_const)
                    self.Notes[idx].q_max = q_note
                else:
                    play_state = "on_falling"
                    # note.play(self.Notes[idx].q_max*q_conv_const)
                    note.set_volume(self.Notes[idx].q_max*q_conv_const)
                    print("idx : {}, name: {}, q: {}, volume: {}".format(note.index, note.name, q_note, self.Notes[idx].q_max*q_conv_const))
                    
            elif play_state == "on_falling":
                self.Notes[idx].t_on+=time_step

                if q_note< q_off_threshold:
                    # note.set_volume(self.Notes[idx].q_max*0.8*q_conv_const)
                    # note.stop(self.Notes[idx].t_on*10)
                    print("idx : {}, name: {}, t_on: {}".format(note.index, note.name, self.Notes[idx].t_on))
                    note.set_volume(self.Notes[idx].q_max*q_conv_const)
                    # self.Notes[idx].t_off += time_step
                    self.Notes[idx].q_max = 0
                    play_state = "turning_off"

            elif play_state =="turning_off":
                self.Notes[idx].t_off += time_step
                # note.set_volume(self.Notes[idx].q_max*q_conv_const * (1-(self.Notes[idx].t_off/self.Notes[idx].t_on*4)**2))

                if self.Notes[idx].t_off>= self.Notes[idx].t_on*1.8:

                    print("idx : {}, name: {}, t_off: {}, t_on: {}".format(note.index, note.name, self.Notes[idx].t_on, self.Notes[idx].t_off))
                    
                    note.fadeout(int(self.Notes[idx].t_off*1e3))    
                    # note.stop()
                    print("fadeout : {}".format(int(self.Notes[idx].t_off*1e3)))
                    
                    self.Notes[idx].t_on = 0
                    self.Notes[idx].t_off = 0
                    play_state = "off"
                
                elif q_note >=q_off_threshold-1e-5:
                    print("Estop idx : {}, name: {}, t_on: {}, t_off: {}".format(note.index, note.name, self.Notes[idx].t_on, self.Notes[idx].t_off))

                    note.fadeout(int(self.Notes[idx].t_on*1e3))   
                    self.Notes[idx].t_on = 0
                    self.Notes[idx].t_off = 0
                    play_state = "off"
                    # note.stop()
                    
            self.Notes[idx].play_state = play_state

        def update_note_w(idx):
            note = self.Notes[idx]
            q_note = q_piano[idx,0]
            w_note = w_piano[idx,0]
            
            play_state = note.play_state
            
            if play_state =="off":

                if q_note >q_on_threshold:
                    note.play(w_note*w_conv_const)
                    print("idx : {}, name: {}, q: {}, volume: {}".format(note.index, note.name, q_note, w_note*w_conv_const))
                    self.Notes[idx].q_max = q_note
                    play_state = "on_rising"
            
            elif play_state == "on_rising":
                self.Notes[idx].t_on+=time_step

                if self.Notes[idx].q_max < q_note+1e-5:
                    self.Notes[idx].q_max = q_note
                else:
                    play_state = "on_falling"
                    print("idx : {}, name: {}, q: {}, volume: {}".format(note.index, note.name, q_note, self.Notes[idx].q_max*q_conv_const))
                    
            elif play_state == "on_falling":
                self.Notes[idx].t_on+=time_step

                if q_note< q_off_threshold:
                    print("idx : {}, name: {}, t_on: {}".format(note.index, note.name, self.Notes[idx].t_on))
                    # note.set_volume(self.Notes[idx].q_max*q_conv_const)
                    self.Notes[idx].q_max = 0
                    play_state = "turning_off"

            elif play_state =="turning_off":
                self.Notes[idx].t_off += time_step
                # note.set_volume(self.Notes[idx].q_max*q_conv_const * (1-(self.Notes[idx].t_off/self.Notes[idx].t_on*4)**2))

                if self.Notes[idx].t_off>= self.Notes[idx].t_on*1.7:

                    print("idx : {}, name: {}, t_off: {}, t_on: {}".format(note.index, note.name, self.Notes[idx].t_on, self.Notes[idx].t_off))
                    
                    note.fadeout(int(self.Notes[idx].t_off*5*1e3))    
                    # note.stop()
                    print("fadeout : {}".format(int(self.Notes[idx].t_off*1e3)))
                    
                    self.Notes[idx].t_on = 0
                    self.Notes[idx].t_off = 0
                    play_state = "off"
                
                elif q_note >=q_off_threshold-1e-5:
                    print("Estop idx : {}, name: {}, t_on: {}, t_off: {}".format(note.index, note.name, self.Notes[idx].t_on, self.Notes[idx].t_off))

                    note.fadeout(int(self.Notes[idx].t_off*5*1e3))    
                    self.Notes[idx].t_on = 0
                    self.Notes[idx].t_off = 0
                    play_state = "off"
                    # note.stop()

            self.Notes[idx].play_state = play_state


        for i in range(self.N_key):
            # update_note(i)
            update_note_w(i)


class Piano_Project():
    def __init__(self, sim = None,q_iiwa_traj=None, q_hand_traj=None, q_all_traj=None, w_iiwa_traj=None, w_hand_traj=None,w_all_traj=None,c_hand_traj = None):
        self.builder = DiagramBuilder()
        self.timestep = 1e-5
        
        if sim != "static":
            self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step=self.timestep)
        else:
            self.plant = MultibodyPlant(time_step = self.timestep)
            self.scene_graph = self.builder.AddSystem(SceneGraph())
            self.plant.RegisterAsSourceForSceneGraph(self.scene_graph)

        self.sim = sim
        self.mu = 1
        
        self.build_plant()

        self.initialize_plant_before_sim()
        self.initialize_simulation()
        self.build_system(sim, q_iiwa_traj, q_hand_traj,q_all_traj,w_iiwa_traj, w_hand_traj,w_all_traj,c_hand_traj)
        self.setup_plant_after_sim()


            
    # def init_meshcat(self):
    #     self.meshcat = StartMeshcat()
    
    def build_plant(self):
        def build_ground():
            ground = AddShape(self.plant, pydrake.geometry.Box(10,10,2.0), name="ground", mu=self.mu)
            self.plant.WeldFrames(self.plant.world_frame(), self.plant.GetFrameByName("ground"), RigidTransform(p=[0,0,-1.0]))
            self.plant.AddFrame(FixedOffsetFrame("frame_top_ground",
                                self.plant.GetFrameByName("ground"), 
                                RigidTransform(p=[0,0,1])))

        def build_robot():
            def get_robot_file(description):
                # Note: I could download remote model resources here if necessary.
                if description == "Kuka LBR iiwa 7":
                    return FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
                elif description == "Kuka LBR iiwa 14":
                    return FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf")
                elif description == "Kinova Jaco Gen2 (7 DoF)":
                    return FindResourceOrThrow("drake/manipulation/models/jaco_description/urdf/j2s7s300.urdf")
                elif description == "Franka Emika Panda":
                    return FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/panda_arm_hand.urdf")
                raise Exception("Unknown Robot model")

            def get_hand_file(description):
                # Note: I could download remote model resources here if necessary.
                if description == "allegro_hand_description_right":
                    return FindResourceOrThrow("drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_right.sdf")
                elif description == "allegro_hand_description_left":
                    return FindResourceOrThrow("drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_left.sdf")
                raise Exception("Unknown Hand model")


            robot_arm = "Kuka LBR iiwa 7"
            robot_hand = "allegro_hand_description_right"

            self.model_arm = Parser(self.plant, self.scene_graph).AddModelFromFile(get_robot_file(robot_arm))
            self.plant.WeldFrames(self.plant.GetFrameByName("frame_top_ground"), 
                    self.plant.get_body(self.plant.GetBodyIndices(self.model_arm)[0]).body_frame(),RigidTransform(p=[0,0,0.0001]))
            
            self.hand_frame = self.plant.get_body(
                            self.plant.GetBodyIndices(self.model_arm)[0]).body_frame()

            self.model_hand = Parser(self.plant, self.scene_graph).AddModelFromFile(get_hand_file(robot_hand))

            self.plant.AddFrame(FixedOffsetFrame("frame_palm",
                                self.plant.GetFrameByName("hand_root"), 
                                RigidTransform(p=[0,0,0.05])))

            self.plant.WeldFrames(self.plant.get_body(self.plant.GetBodyIndices(self.model_arm)[7]).body_frame(),
                            self.plant.get_body(self.plant.GetBodyIndices(self.model_hand)[0]).body_frame(),
                            RigidTransform(p=[0,0,0.05]))

            self.finger_link_list = ["link_15", "link_3", "link_7","link_11"]
            self.finger_connect_list = ["link_14", "link_2", "link_6","link_10"]
            
            for body_name in self.finger_link_list:
                self.plant.AddFrame(FixedOffsetFrame("frame_fingercontact_"+body_name,
                        self.plant.GetFrameByName(body_name),
                        RigidTransform(p=[0,0,0.032])))

        def build_piano():
            color_black = [0, 0, 0, 1]
            color_white = [1,1,1,1]
            color_base = [0.3,0.2,0.1,1]           

            self.base_name = "piano_base"
            self.base_size = [1.8, 0.5, 0.2]
            self.base_pos = [-0.20, 0.9, 0]

            key_scale = 2.2
            key_size = np.array([0.019, 0.04, 0.02])*key_scale
            key_offset = np.array([0.0042, 0.0,0])*key_scale

            key_minor_size = key_size
            key_minor_size[0] *=0.75
            key_minor_size[1] *=1

            self.key_size = key_size
            
            keyboard_base = AddShape(self.plant, pydrake.geometry.Box(self.base_size[0],self.base_size[1],self.base_size[2]), name=self.base_name, mass=10, mu=self.mu, color=color_base)
           
            self.plant.AddFrame(FixedOffsetFrame("frame_bottom_"+self.base_name,
                    self.plant.GetFrameByName(self.base_name), 
                    RigidTransform(p=[0,0,-self.base_size[2]*0.5])))

            self.plant.AddFrame(FixedOffsetFrame("frame_top_"+self.base_name,
                                self.plant.GetFrameByName(self.base_name), 
                                RigidTransform(p=[0,0,self.base_size[2]*0.5])))

            self.plant.WeldFrames(self.plant.GetFrameByName("frame_top_ground"), 
                                    self.plant.GetFrameByName("frame_bottom_"+self.base_name), 
                                    RigidTransform(p=self.base_pos))

            def add_key_spring(name,key_pos, key_size,color = color_white):
                key_mass= 0.01
                key_base_name = "key_anchor_"+name
                key_name = "key_"+name
                key_joint_name = "key_joint_"+name

                
                key_base = AddShape(self.plant, 
                                    pydrake.geometry.Box(key_size[0],key_size[1],key_size[2]*0.1), 
                                    name=key_base_name, 
                                    mass=1, 
                                    mu=1, 
                                    color=color)

                self.plant.AddFrame(FixedOffsetFrame("frame_bottom_"+key_base_name,
                                    self.plant.GetFrameByName(key_base_name), 
                                    RigidTransform(p=[0,0,-key_size[2]*0.05])))
                self.plant.AddFrame(FixedOffsetFrame("frame_top_"+key_base_name,
                                    self.plant.GetFrameByName(key_base_name), 
                                    RigidTransform(p=[0,0,+key_size[2]*0.05+0.05])))

                self.plant.WeldFrames(self.plant.GetFrameByName("frame_top_"+self.base_name), 
                                    self.plant.GetFrameByName("frame_bottom_"+key_base_name), 
                                    RigidTransform(p=key_pos))
                
                
                key = AddShape(self.plant, 
                                    pydrake.geometry.Box(key_size[0],key_size[1],key_size[2]), 
                                    name=key_name, 
                                    mass=key_mass, 
                                    mu=self.mu, 
                                    color=color)

                self.plant.AddFrame(FixedOffsetFrame("frame_bottom_"+key_name,
                                    self.plant.GetFrameByName(key_name), 
                                    RigidTransform(p=[0,0,-key_size[2]*0.5])))

                X_KT = RigidTransform(p=[0,0,+key_size[2]*0.3])
                X_KR1 = RigidTransform(R=RotationMatrix.MakeYRotation(np.pi))
                X_KR2 = RigidTransform(R=RotationMatrix.MakeZRotation(-np.pi/2))
                X_FContact = X_KT.multiply(X_KR1.multiply(X_KR2))

                self.plant.AddFrame(FixedOffsetFrame("frame_keycontact_"+key_name,
                                    self.plant.GetFrameByName(key_name),
                                    X_FContact))
                                    
                key_Joint = self.plant.AddJoint(
                                    PrismaticJoint(name = key_joint_name,
                                    frame_on_parent=self.plant.GetFrameByName("frame_top_"+key_base_name),
                                    frame_on_child=self.plant.GetFrameByName("frame_bottom_"+key_name),
                                    axis=[0,0,1],
                                    pos_upper_limit = 0.1,
                                    pos_lower_limit = 0.01,
                                    damping=0))

                key_spring = self.plant.AddForceElement(
                            LinearSpringDamper( bodyA = self.plant.GetBodyByName(key_base_name),
                                                p_AP = [0,0,key_size[2]*0.1*0.5],
                                                bodyB = self.plant.GetBodyByName(key_name),
                                                p_BQ = [0,0,-key_size[2]*0.5],
                                                free_length = 0.03,
                                                stiffness = 0.01,
                                                damping = 0.0001)            )    
                return

            def add_key_hinge(name,key_pos, key_size,color = color_white):
                key_mass= 0.1
                key_base_name = "key_anchor_"+name
                key_name = "key_"+name
                key_joint_name = "key_joint_"+name

                key_moment_arm = key_size[1]*7
                color_key_base = [0.4,0.2,0.1,1]
                key_base = AddShape(self.plant, 
                                    pydrake.geometry.Box(key_size[0],key_size[0]*0.5,key_size[2]*0.1), 
                                    name=key_base_name, 
                                    mass=1, 
                                    mu=1, 
                                    color=color_key_base)

                self.plant.AddFrame(FixedOffsetFrame("frame_bottom_"+key_base_name,
                                    self.plant.GetFrameByName(key_base_name), 
                                    RigidTransform(p=[0,0,-key_size[2]*0.05])))
                self.plant.AddFrame(FixedOffsetFrame("frame_top_"+key_base_name,
                                    self.plant.GetFrameByName(key_base_name), 
                                    RigidTransform(p=[0,0,+key_size[2]*0.05+0.05])))

                hinge_offset_constant = np.array([0, key_moment_arm ,key_size[2]*2])
                p_key_hinge = RigidTransform(p=hinge_offset_constant+key_pos)

                self.plant.WeldFrames(self.plant.GetFrameByName("frame_top_"+self.base_name), 
                                    self.plant.GetFrameByName("frame_bottom_"+key_base_name), 
                                    p_key_hinge)
                
                
                key = AddShape(self.plant, 
                                    pydrake.geometry.Box(key_size[0],key_size[1],key_size[2]), 
                                    name=key_name, 
                                    mass=key_mass, 
                                    mu=self.mu, 
                                    color=color)

                self.plant.AddFrame(FixedOffsetFrame("frame_rev_"+key_name,
                                    self.plant.GetFrameByName(key_name), 
                                    RigidTransform(p=[0,key_moment_arm,0])))

                X_KT = RigidTransform(p=[0,0,key_size[2]*0.25])
                X_KR1 = RigidTransform(R=RotationMatrix.MakeYRotation(np.pi))
                X_KR2 = RigidTransform(R=RotationMatrix.MakeZRotation(-np.pi/2))
                X_FContact = X_KT.multiply(X_KR1.multiply(X_KR2))

                self.plant.AddFrame(FixedOffsetFrame("frame_keycontact_"+key_name,
                                    self.plant.GetFrameByName(key_name),
                                    X_FContact))
                        
                key_revolute_joint = self.plant.AddJoint(RevoluteJoint(
                        name="revolve_joint_"+key_name, 
                        frame_on_parent=self.plant.GetFrameByName(key_base_name),
                        frame_on_child=self.plant.GetFrameByName("frame_rev_"+key_name), 
                        axis=[1, 0, 0],
                        damping=0.0))    

                door_hinge_config = DoorHingeConfig()
                door_hinge_config.spring_constant = 5
                door_hinge_config.spring_zero_angle_rad=1#key_size[2]/key_moment_arm*1.1
                door_hinge_config.catch_torque =door_hinge_config.spring_constant*1.55#door_hinge_config.spring_constant #door_hinge_config.spring_constant
                door_hinge_config.catch_width = door_hinge_config.spring_zero_angle_rad#1#door_hinge_config.spring_zero_angle_rad #door_hinge_config.spring_zero_angle_rad*0.9
                
                door_hinge_config.viscous_friction  = door_hinge_config.spring_constant*0.2#door_hinge_config.spring_constant*0.1
                door_hinge_config.static_friction_torque =door_hinge_config.spring_constant*0.01#0.01
                door_hinge_config.dynamic_friction_torque = door_hinge_config.spring_constant*0.0#door_hinge_config.spring_constant*0.01
                door_hinge = self.plant.AddForceElement(DoorHinge(
                        joint=key_revolute_joint, config=door_hinge_config))
                return


            key_major_list = ["C","D","E","F","G","A","B"]
            key_minor_list = ["C#","D#","E#","F#","G#","A#","B#"]
            N_scale = 3

            key_major_pos = np.array([-(key_size[0]+key_offset[0])*(7*N_scale/2)*2+0.2,-self.base_size[1]*0.5,0])
            
            self.key_list =[]
            N_scale += 1 # Add one more octave 
            N_scale += 1 # Add one more octave 

            for s in range(N_scale):
                for i in range(7):
                    key_name = key_major_list[i]+"_{}".format(s)
                    # add_key_spring(key_name, key_major_pos, key_size)
                    add_key_hinge(key_name, key_major_pos, key_size)
                    self.key_list.append(key_name)

                    if i==0 or i==1 or i==3 or i==4 or i==5:
                        key_minor_name = key_minor_list[i]+"_{}".format(s)
                        key_minor_pos = np.copy(key_major_pos)
                        key_minor_pos[0] += (key_size[0]+key_offset[0])*0.5
                        key_minor_pos[1] += (key_size[1]+0.005)
                        key_minor_pos[2] += key_size[2]*0.5

                        add_key_hinge(key_minor_name, key_minor_pos, key_minor_size,color=color_black)
                        
                        self.key_list.append(key_minor_name)
                        

                    key_major_pos[0] += key_size[0]+key_offset[0]
            return

        build_ground()
        build_piano()
        build_robot()
        self.plant.mutable_gravity_field().set_gravity_vector([0, 0, 0]) # easy trick for gravity compensation
        self.plant.Finalize()

    def initialize_plant_before_sim(self):
        def initialize_robot():
            # self.finger_link_list = ["link_11", "link_7", "link_3","link_15"]
            # self.connect_link_list = ["link_14", "link_2", "link_6","link_10"]


            # def add_triad(link_names):
            #     for body_name in link_names:
            #         AddMultibodyTriad(self.plant.GetFrameByName(body_name), self.scene_graph,length=0.05)

            # add_triad(self.finger_link_list)            
            self.fingers = []
            self.finger_frames = []
            self.finger_link_frames = []

            self.palm_frame = self.plant.GetFrameByName("frame_palm")
            # AddMultibodyTriad(self.palm_frame , self.scene_graph,length=0.05)
            
            for body_name in self.finger_link_list:
                self.fingers.append(self.plant.GetBodyByName(body_name))
                # self.finger_frames.append(self.plant.GetFrameByName(body_name))
                self.finger_frames.append(self.plant.GetFrameByName("frame_fingercontact_"+body_name))
                # AddMultibodyTriad(self.plant.GetFrameByName("frame_fingercontact_"+body_name), self.scene_graph,length=0.01)

            for body_name in self.finger_connect_list:
                self.finger_link_frames.append(self.plant.GetFrameByName(body_name))
                # AddMultibodyTriad(self.plant.GetFrameByName(body_name,self.model_hand), self.scene_graph,length=0.05)
        
        def initialize_piano():
            for key in self.key_list:
                frame_name = "frame_keycontact_key_"+key
                key_frame = self.plant.GetFrameByName(frame_name)
                #ddMultibodyTriad(key_frame , self.scene_graph,length=0.05)


        initialize_robot()
        initialize_piano()

    def setup_plant_after_sim(self):
        def setup_piano():
            for key in self.key_list:
                frame_name = "frame_keycontact_key_"+key
                key_frame = self.plant.GetFrameByName(frame_name)

                key_pose = self.plant.CalcRelativeTransform(
                            self.plant_context,
                            frame_A=self.plant.GetFrameByName("frame_top_ground"),
                            frame_B=key_frame)
                            
                # AddMeshcatTriad(meshcat, "meshcat_" + frame_name,
                #             length=0.015, radius=0.006, X_PT=key_pose)
                            

            self.note_dictionary =dict.fromkeys(self.key_list)
            # self.music_note_list = []

            for idx, key in enumerate(self.key_list):
                key_pose = self.plant.CalcRelativeTransform(
                            self.plant_context,
                            frame_A=self.plant.GetFrameByName("frame_top_ground"),
                            frame_B=self.plant.GetFrameByName("frame_keycontact_key_"+key))
                pos = key_pose.translation()
                rotation = key_pose.rotation()

                key_frame = self.plant.GetFrameByName("frame_keycontact_key_"+key)
                
                k_type = idx%12 
                if k_type == 1 or k_type ==3 or k_type ==6 or k_type ==8 or k_type==10:
                    key_type = "sharp"
                else:
                    key_type = "flat"
            
                self.note_dictionary[key] = Keyboard_Key(key, pos=pos,rotation = rotation,frame = key_frame,keytype =key_type)
            


            first_key = self.key_list[0]
            first_key_pose = self.plant.CalcRelativeTransform(
                            self.plant_context,
                            frame_A=self.plant.GetFrameByName("frame_top_ground"),
                            frame_B=self.plant.GetFrameByName("key_"+first_key))
            first_key_pos = first_key_pose.translation()
            
            last_key = self.key_list[-1]
            last_key_pose = self.plant.CalcRelativeTransform(
                            self.plant_context,
                            frame_A=self.plant.GetFrameByName("frame_top_ground"),
                            frame_B=self.plant.GetFrameByName("key_"+last_key))
            last_key_pos = first_key_pose.translation()
            
            self.music_note_list = list(self.note_dictionary)

            return

        if self.sim != "static":
            setup_piano()       


    def get_finger_pose(self, index=0):
        pose = RigidTransform()
        
        if index <4:
            pose = self.plant.EvalBodyPoseInWorld(self.plant_context, self.fingers[index])
        
        return pose

    def initialize_simulation(self):
        self.visualizer = MeshcatVisualizerCpp.AddToBuilder(self.builder, self.scene_graph, meshcat)
        
        
    def build_system(self, sim = None, q_iiwa_traj=None, q_hand_traj=None, q_all_traj = None, w_iiwa_traj=None, w_hand_traj=None, w_all_traj=None, c_hand_traj=None):

        def connect_controller():
            self.robot_controller = self.builder.AddSystem(MyControllerSystem(
                                                        self.plant))

            self.builder.Connect(self.plant.GetOutputPort("iiwa7_continuous_state"),
                                    self.robot_controller.GetInputPort("iiwa7_state"))
            self.builder.Connect(self.plant.GetOutputPort("allegro_hand_right_continuous_state"),
                                    self.robot_controller.GetInputPort("hand_state"))

            # self.builder.Connect(self.plant.GetOutputPort("allegro_hand_right_generalized_contact_forces"),
            #                         self.robot_controller.GetInputPort("hand_contact_forces"))
                                    
            # self.builder.Connect(self.plant.GetOutputPort("contact_results"),
            #                     self.robot_controller.GetInputPort("contact_result"))

            self.builder.Connect(self.robot_controller.GetOutputPort("iiwa7_torque"),
                                    self.plant.GetInputPort("iiwa7_actuation"))
            self.builder.Connect(self.robot_controller.GetOutputPort("hand_torque"),
                                    self.plant.GetInputPort("allegro_hand_right_actuation"))
            
        

        def add_trajectory_to_controller(q_iiwa_traj, q_hand_traj,w_iiwa_traj,w_hand_traj,c_hand_traj):
            q_iiwa_source = self.builder.AddSystem(TrajectorySource(q_iiwa_traj))
            q_iiwa_source.set_name("q_iiwa_traj")
            q_hand_source = self.builder.AddSystem(TrajectorySource(q_hand_traj))
            q_hand_source.set_name("q_hand_traj")
            
            self.builder.Connect(q_iiwa_source.get_output_port(),
                                    self.robot_controller.GetInputPort("iiwa7_des_pos"))
            self.builder.Connect(q_hand_source.get_output_port(),
                                    self.robot_controller.GetInputPort("hand_des_pos"))


            w_iiwa_source = self.builder.AddSystem(TrajectorySource(w_iiwa_traj))
            w_iiwa_source.set_name("w_iiwa_traj")
            self.builder.Connect(w_iiwa_source.get_output_port(),
                    self.robot_controller.GetInputPort("iiwa7_des_vel"))


            w_hand_source = self.builder.AddSystem(TrajectorySource(w_hand_traj))
            w_hand_source.set_name("w_hand_traj")
            self.builder.Connect(w_hand_source.get_output_port(),
                    self.robot_controller.GetInputPort("hand_des_vel"))

            c_hand_source = self.builder.AddSystem(TrajectorySource(c_hand_traj))
            c_hand_source.set_name("c_hand_traj")
            self.builder.Connect(c_hand_source.get_output_port(),
                    self.robot_controller.GetInputPort("finger_des_contact"))


        def add_geomtery_generator(q_all_traj):
            q_all_source = self.builder.AddSystem(TrajectorySource(q_all_traj))
            q_all_source.set_name("q_all_source")
            
            w_all_source = self.builder.AddSystem(TrajectorySource(w_all_traj))
            w_all_source.set_name("w_all_source")


            self.piano_sound = self.builder.AddSystem(PianoOutputSystem(self.plant,self.key_list))
            self.builder.Connect(q_all_source.get_output_port(), self.piano_sound.GetInputPort("q"))
            self.builder.Connect(w_all_source.get_output_port(), self.piano_sound.GetInputPort("w"))
            
            self.piano_sound.set_name("piano_sound_generator")
            
            self.robot_pose = self.builder.AddSystem(MultibodyPositionToGeometryPose(self.plant))

            self.builder.Connect(self.piano_sound.GetOutputPort("state_output"), self.robot_pose.get_input_port())
            self.builder.Connect(   self.robot_pose.get_output_port(),
                                self.scene_graph.get_source_pose_port(self.plant.get_source_id()))


        def add_data_logger():
                self.q_logger = LogVectorOutput(self.plant.GetOutputPort("continuous_state"), self.builder)
                self.q_iiwa_logger = LogVectorOutput(self.plant.GetOutputPort("iiwa7_continuous_state"), self.builder)
                self.q_hand_logger = LogVectorOutput(self.plant.GetOutputPort("allegro_hand_right_continuous_state"), self.builder)
                
        if sim == "dynamic":
            connect_controller()
            add_trajectory_to_controller(q_iiwa_traj, q_hand_traj,w_iiwa_traj,w_hand_traj,c_hand_traj)
            add_data_logger()

        elif sim == "static":
            add_geomtery_generator(q_all_traj)

        else: #sim is None
            connect_controller()

        self.diagram = self.builder.Build()
        self.context = self.diagram.CreateDefaultContext()
        self.plant_context =  self.plant.GetMyMutableContextFromRoot(self.context)

    def forward_kinematics_demo(self):
        def my_callback(context):

            self._iiwa = self.plant.GetModelInstanceByName("iiwa7")
            self._hand = self.plant.GetModelInstanceByName("allegro_hand_right")
            self.iiwa_indices = self.plant.GetJointIndices(self._iiwa)
            self.hand_indices = self.plant.GetJointIndices(self._hand)

            qqq = self.plant.GetOutputPort("continuous_state").Eval(self.plant_context)

            q =self.plant.GetPositions(self.plant_context)
            dq =self.plant.GetPositions(self.plant_context)

            self._W = self.plant.world_frame()

            finger_name_list = ["link_15", "link_3", "link_7","link_11"]
            _F0 = self.plant.GetFrameByName("frame_fingercontact_"+finger_name_list[0])
            _P = self.plant.GetFrameByName("frame_palm")
            _W = self.plant.world_frame()

            J_G = self.plant.CalcJacobianSpatialVelocity(
                self.plant_context, JacobianWrtVariable.kV, 
                _F0, [0,0,0], _W, _W)

            print(np.shape(qqq))
            print(np.shape(J_G))
            clear_output(wait=True)
        sliders = MeshcatJointSliders(meshcat, self.plant, self.context)
        sliders.Run(self.visualizer, self.context,my_callback)
        meshcat.DeleteAddedControls()
        meshcat.Delete()
    
    def run_simulation_demo(self, q0_init=np.zeros((71+12,1)),play_rate =1,t_duration = 40):
        
        hand_indices =np.array([43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58])+12+12
        
        q0_init[hand_indices] = 0

        if self.sim =="dynamic":
            q0 = self.plant.GetPositions(self.plant_context)
            _,_,_,q_all = self.ik_init(q0)
            self.plant.SetPositions(self.plant_context, q_all)
            self.plant.SetPositions(self.plant_context, q0_init)

        
        simulator = Simulator(self.diagram, self.context)
        context = simulator.get_mutable_context()

        simulator.set_target_realtime_rate(play_rate)
        duration = t_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)

        meshcat.DeleteAddedControls()
        meshcat.Delete()
        
        if self.sim =="dynamic":
            log = self.q_logger.FindMutableLog(context)
            return log
    

    def inverse_kinematics_demo(self):

        q0 = self.plant.GetPositions(self.plant_context)
        finger_frame = self.plant.GetFrameByName("link_7",self.model_hand)

        def my_callback(context, pose):
            ik = InverseKinematics(self.plant, self.plant_context)
            ik.AddPositionConstraint(
                finger_frame, [0, 0, 0], self.plant.world_frame(),
                pose.translation(), pose.translation())
            # ik.AddOrientationConstraint(
            #     gripper_frame, RotationMatrix(), plant.world_frame(),
            #     pose.rotation(), 0.0)
            # ik.AddMinimumDistanceConstraint(0.001, 0.1)
            prog = ik.get_mutable_prog()
            q = ik.q()
            # prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
            prog.SetInitialGuess(q, q0)
            result = Solve(ik.prog())
            
            # clear_output(wait=True)

            # if result.is_success():
            #     print("IK success")
            # else:
            #     print("IK failure")

        sliders = MeshcatPoseSliders(meshcat)
        sliders.SetPose(self.plant.EvalBodyPoseInWorld(
            self.plant_context, self.plant.GetBodyByName("link_7",self.model_hand)))

        # set the initial z lower, to make the interaction interesting.
        # sliders.SetXyz([0.4, 0.2, 0.65])
        sliders.Run(self.visualizer, self.context, my_callback)
        meshcat.DeleteAddedControls()
        meshcat.Delete()

        return
    def visualize_kinematic_tree(self):  
        display(SVG(pydot.graph_from_dot_data(self.plant.GetTopologyGraphvizString())[0].create_svg()))

    def visualize_diagram(self):  
        display(SVG(pydot.graph_from_dot_data(self.diagram.GetGraphvizString())[0].create_svg()))
    
    def test_finger_planning(self):
        tchord = [self.music_note_list[0],self.music_note_list[4], self.music_note_list[7]]

        test_music_notes = [self.note_dictionary[tchord[0]], self.note_dictionary[tchord[1]], self.note_dictionary[tchord[2]]]

        n, fidx,cost = self.find_finger_correspondence(test_music_notes)
        print(self.find_finger_correspondence(test_music_notes))

    def find_finger_correspondence(self,key_list):
        N_finger =4
        f_pos = self.get_finger_pose(0).translation()
        for i in range(N_finger-1):
            f_pos = np.vstack((f_pos,self.get_finger_pose(i+1).translation()))

        N_key = len(key_list)
        indices = np.empty(N_key,dtype = int)
        
        if N_key >= 4:
            fidx = np.zeros((1,4))
            cost = np.zeros((1,1))
            
            N_sol = 1
            fidx[0,:] = [0,1,2,3]
            cost[0,0] = 1

        elif N_key <=0:
            N_sol = 0
            fidx = -1
            cost = -1

        else:
            dist =np.zeros((N_key, N_finger))
            for key_idx,key in enumerate(key_list):
                for f in range(N_finger):
                    note = self.note_dictionary[key]
                    p_FK = note.pos - self.get_finger_pose(f).translation()
                    d_FK = np.dot(p_FK[:2], p_FK[:2]) # only consider x and y 
                    dist[key_idx,f] = d_FK

            if N_key ==1: 
                N_sol = 3
                fidx = np.zeros((N_sol,N_key))
                cost = np.zeros((N_sol,1))   
                i_sol = 0                 
                for f in range(1,N_finger):
                    fidx[i_sol,0] = f
                    cost[i_sol,0] = dist[0,f]
                    i_sol+=1

            elif N_key ==2:
                N_sol = 6
                fidx = np.zeros((N_sol,N_key))
                cost = np.zeros((N_sol,1))    

                i_sol = 0
                for f1 in range(N_finger):
                    for f2 in range(f1+1,N_finger):
                        fidx[i_sol,:] = [f1,f2]
                        cost[i_sol,0] = dist[0,f1]+dist[0,f2]
                        i_sol+=1

            elif N_key ==3:
                N_sol = 4
                fidx = np.zeros((N_sol,N_key))
                cost = np.zeros((N_sol,1))    

                i_sol = 0
                for f1 in range(N_finger):
                    for f2 in range(f1+1,N_finger):
                        for f3 in range(f2+1,N_finger):
                            fidx[i_sol,:] = [f1,f2,f3]
                            cost[i_sol,0] = dist[0,f1]+dist[0,f2]+dist[0,f3]
                            i_sol+=1                                
            
            

        return N_sol, fidx, cost

    def ik_init(self, q0):

        ik = InverseKinematics(self.plant, self.plant_context)

        T_center = self.note_dictionary[self.key_list[18]].pos

        T_const_palm_lower = np.array([-0.01,-self.key_size[1]*4-0.01,self.key_size[2]*3])
        T_const_palm_upper = np.array([+0.01,-self.key_size[1]*2+0.01,self.key_size[2]*5+0.01])

        T_palm_lower =T_center + T_const_palm_lower
        T_palm_upper =T_center + T_const_palm_upper


        c = ik.AddPositionConstraint(
            self.palm_frame , [0, 0, 0], self.plant.world_frame(),
            T_palm_lower, T_palm_upper)

        c = ik.AddAngleBetweenVectorsConstraint(
            self.palm_frame, [0, 0, 1],
            self.plant.world_frame(), [0,1,0],
            0, np.pi/2*0.05
        )
        c = ik.AddAngleBetweenVectorsConstraint(
            self.palm_frame, [1, 0, 0],
            self.plant.world_frame(), [0,0,-1],
            0, np.pi/2*0.05
        )               
        c = ik.AddAngleBetweenVectorsConstraint(
            self.palm_frame, [0, 1, 0],
            self.plant.world_frame(), [-1,0,0],
            0, np.pi/2*0.05
        )               
           
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = Solve(ik.prog())

        q_iiwa = self.plant.GetPositions(self.plant_context, 
                    self.plant.GetModelInstanceByName("iiwa7"))
        q_hand = self.plant.GetPositions(self.plant_context, 
                    self.plant.GetModelInstanceByName("allegro_hand_right"))  

        q_all = result.GetSolution()
        return result.is_success(), q_iiwa, q_hand, q_all

    def ik_prepress_chord(self, q0, note_on_fingers):

        ik = InverseKinematics(self.plant, self.plant_context)

        T_finger = []
        T_link = []

        for i in range(4):
            T_finger.append( self.plant.CalcRelativeTransform(
                                self.plant_context,
                                frame_A=self.plant.GetFrameByName("frame_top_ground"),
                                frame_B=self.finger_frames[i]).translation())

            T_link.append( self.plant.CalcRelativeTransform(
                                self.plant_context,
                                frame_A=self.plant.GetFrameByName("frame_top_ground"),
                                frame_B=self.finger_link_frames[i]).translation())

        T_palm = np.copy(self.plant.CalcRelativeTransform(
                                self.plant_context,
                                frame_A=self.plant.GetFrameByName("frame_top_ground"),
                                frame_B=self.palm_frame).translation())
                                
        T_const_finger_tip_offset = np.array([0,0,self.key_size[2]*1.6])

        T_const_fingertip_flat_lower = np.array([-self.key_size[0]*0.15,-self.key_size[1]*0.4,-self.key_size[2]*0.01])
        T_const_fingertip_flat_upper = np.array([+self.key_size[0]*0.15,+self.key_size[1]*0.15,+self.key_size[2]])

        T_const_fingertip_sharp_lower = np.array([-self.key_size[0]*0.15,-self.key_size[1]*0.3,-self.key_size[2]*0.01])
        T_const_fingertip_sharp_upper = np.array([+self.key_size[0]*0.15,+self.key_size[1]*0.2,+self.key_size[2]])

        T_const_fingertip_none_lower = np.array([-self.key_size[0],-self.key_size[1]*0.5,self.key_size[2]*0.01])
        T_const_fingertip_none_upper = np.array([+self.key_size[0],+self.key_size[1]*0.5,+self.key_size[2]*1.5])
        
        finger_cnt = 0



        T_const_fingertip_flat_lower2 = np.array([-self.key_size[0]*0.15,-self.key_size[1]*0.6,-self.key_size[2]*0.5])
        T_const_fingertip_flat_upper2 = np.array([+self.key_size[0]*0.3,+self.key_size[1]*0.1,+self.key_size[2]*2])

        T_const_fingertip_sharp_lower2 = np.array([-self.key_size[0]*0.3,-self.key_size[1]*0.5,-self.key_size[2]*0.5])
        T_const_fingertip_sharp_upper2 = np.array([+self.key_size[0]*0.3,+self.key_size[1]*0.3,+self.key_size[2]*2])


        for idx, note in enumerate(note_on_fingers):
            T_key = np.copy(T_finger[idx])
            T_keylink = np.copy(T_link[idx])

            
            if note != None and "none" in note:
                
                T_lower = T_key + T_const_fingertip_none_lower
                T_upper = T_key + T_const_fingertip_none_upper

                T_link_lower = T_keylink + T_const_fingertip_none_lower
                T_linK_upper = T_keylink + T_const_fingertip_none_upper
            else:
                key_type = self.note_dictionary[note].keytype

                T_key2 = np.copy(self.note_dictionary[note].pos)
                
                if key_type =="flat":
                    T_key += T_const_finger_tip_offset
                    T_keylink += T_const_finger_tip_offset

                    T_lower = T_key + T_const_fingertip_flat_lower
                    T_upper = T_key + T_const_fingertip_flat_upper

                    T_link_lower = T_keylink + T_const_fingertip_none_lower
                    T_linK_upper = T_keylink + T_const_fingertip_none_upper


                    # T_key2 += T_const_finger_tip_offset
                    # T_lower2 = T_key2 + T_const_fingertip_flat_lower2
                    # T_upper2 = T_key2 + T_const_fingertip_flat_upper2

                else :
                    # T_key = self.note_dictionary[note].pos
                    T_key += T_const_finger_tip_offset
                    T_keylink += T_const_finger_tip_offset
                    T_lower = T_key + T_const_fingertip_sharp_lower
                    T_upper = T_key + T_const_fingertip_sharp_upper
                    T_link_lower = T_keylink + T_const_fingertip_none_lower
                    T_linK_upper = T_keylink + T_const_fingertip_none_upper
                
                    # T_key2 += T_const_finger_tip_offset
                    # T_lower2 = T_key2 + T_const_fingertip_sharp_lower2
                    # T_upper2 = T_key2 + T_const_fingertip_sharp_upper2

      
            c = ik.AddPositionConstraint(
                self.finger_frames[idx], [0, 0, 0],self.plant.world_frame(),
                T_lower, T_upper)

      
            # c = ik.AddPositionConstraint(
            #     self.finger_frames[idx], [0, 0, 0],self.plant.world_frame(),
            #     T_lower2, T_upper2)
                
            c = ik.AddPositionConstraint(
                self.finger_link_frames[idx], [0, 0, 0],self.plant.world_frame(),
                T_link_lower, T_linK_upper)

            # if "none" in note:
            #     1

            # else:
            #     if key_type == "flat":
            #         c = ik.AddAngleBetweenVectorsConstraint(
            #             self.finger_frames[idx], [0, 0, 1],
            #             self.plant.world_frame(), [0,0,-1],
            #             0, np.pi/2*0.3
            #         )
            #         c = ik.AddAngleBetweenVectorsConstraint(
            #             self.finger_frames[idx], [0, 0, 1],
            #             self.plant.world_frame(), [1,0,0],
            #             np.pi/2*0.95, np.pi/2*1.05
            #         )

            #         c = ik.AddAngleBetweenVectorsConstraint(
            #             self.finger_frames[idx], [0, 0, 1],
            #             self.plant.world_frame(), [1,0,0],
            #             np.pi/2*0.95, np.pi/2*1.3
            #         )
            #     else:
            #         c = ik.AddAngleBetweenVectorsConstraint(
            #             self.finger_frames[idx], [0, 0, 1],
            #             self.plant.world_frame(), [0,0,-1],
            #             0, np.pi/2*0.7
            #         )
            #         c = ik.AddAngleBetweenVectorsConstraint(
            #             self.finger_frames[idx], [0, 0, 1],
            #             self.plant.world_frame(), [1,0,0],
            #             np.pi/2*0.8, np.pi/2*1.2
            #         )

                    # c = ik.AddAngleBetweenVectorsConstraint(
                    #     self.finger_frames[idx], [0, 0, 1],
                    #     self.plant.world_frame(), [1,0,0],
                    #     np.pi/2*0.95, np.pi/2*1.3       
                    #              

        T_const_palm_lower = np.array([-self.key_size[0]*2,-self.key_size[1]*0.4,-self.key_size[2]*0.3])
        T_const_palm_upper = np.array([+self.key_size[0]*2,+self.key_size[1]*0.45,+self.key_size[2]*2.5])

        T_palm += T_const_finger_tip_offset
        T_palm_lower = T_palm + T_const_palm_lower
        T_palm_upper = T_palm + T_const_palm_upper

        c = ik.AddPositionConstraint(
            self.palm_frame , [0, 0, 0], self.plant.world_frame(),
            T_palm_lower, T_palm_upper)

        c = ik.AddAngleBetweenVectorsConstraint(
            self.palm_frame, [0, 0, 1],
            self.plant.world_frame(), [0,1,0],
            0, np.pi/2*0.67
        )
        # T_const_hand_lower = np.array([-2,-2,self.key_size[2]*0.2])
        # T_const_hand_upper = np.array([+2,2,2])
        # T_hand_lower = T_center + T_const_hand_lower
        # T_hand_upper = T_center + T_const_hand_upper

        # c = ik.AddPositionConstraint(
        #     self.hand_frame, [0, 0, 0],self.plant.world_frame(),
        #     T_hand_lower, T_palm_upper)
                
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = Solve(ik.prog())

        q_iiwa = self.plant.GetPositions(self.plant_context, 
                    self.plant.GetModelInstanceByName("iiwa7"))
        q_hand = self.plant.GetPositions(self.plant_context, 
                    self.plant.GetModelInstanceByName("allegro_hand_right"))  

        return result.is_success(), q_iiwa, q_hand

    def ik_transition(self, q0):

        ik = InverseKinematics(self.plant, self.plant_context)

        T_center = self.note_dictionary[self.key_list[18]].pos
        T_const_release_lower = np.array([-1.3,-1, self.key_size[2]*1.1])
        T_const_release_upper = np.array([1.3,1,self.key_size[2]*10])


        for idx in range(4):
            T_key = T_center
            T_lower = T_key + T_const_release_lower
            T_upper = T_key + T_const_release_upper


            c = ik.AddPositionConstraint(
                self.finger_frames[idx], [0, 0, 0],self.plant.world_frame(),
                T_lower, T_upper)


        T_const_palm_rest_lower = np.array([-1.3,-1,self.key_size[2]*1.2])
        T_const_palm_rest_upper = np.array([+1.3,1,self.key_size[2]*10])

        T_palm_lower =T_key + T_const_palm_rest_lower
        T_palm_upper =T_key + T_const_palm_rest_upper

        c = ik.AddPositionConstraint(
            self.palm_frame , [0, 0, 0], self.plant.world_frame(),
            T_palm_lower, T_palm_upper)

        c = ik.AddAngleBetweenVectorsConstraint(
            self.palm_frame, [0, 0, 1],
            self.plant.world_frame(), [0,1,0],
            0, np.pi/2*0.5
        )
        # T_const_hand_lower = np.array([-2,-2,self.key_size[2]*0.2])
        # T_const_hand_upper = np.array([+2,2,2])
        # T_hand_lower = T_center + T_const_hand_lower
        # T_hand_upper = T_center + T_const_hand_upper

        # c = ik.AddPositionConstraint(
        #     self.hand_frame, [0, 0, 0],self.plant.world_frame(),
        #     T_hand_lower, T_palm_upper)
                
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = Solve(ik.prog())

        q_iiwa = self.plant.GetPositions(self.plant_context, 
                    self.plant.GetModelInstanceByName("iiwa7"))
        q_hand = self.plant.GetPositions(self.plant_context, 
                    self.plant.GetModelInstanceByName("allegro_hand_right"))  

        return result.is_success(), q_iiwa, q_hand

    def ik_transition_arm(self, q_start, q_end, N_list=1):

        ik = InverseKinematics(self.plant, self.plant_context)
        

        def get_pose(frame):
            return self.plant.CalcRelativeTransform(
                                self.plant_context,
                                frame_A=self.plant.GetFrameByName("frame_top_ground"),
                                frame_B=frame)

        self.plant.SetPositions(self.plant_context, q_start)
        X_start_palm = get_pose(self.palm_frame)
        T_start_palm = X_start_palm.translation()
        R_start_palm = X_start_palm.rotation()

        self.plant.SetPositions(self.plant_context, q_end)
        X_end_palm = get_pose(self.palm_frame)
        T_end_palm = X_end_palm.translation()
        R_end_palm = X_end_palm.rotation()
        

        palm_T_traj = PiecewisePolynomial.FirstOrderHold(
            [0.0, 1.0],
            np.vstack([[T_start_palm],
                        [T_end_palm]]).T)

        palm_R_traj = PiecewiseQuaternionSlerp()
        palm_R_traj.Append(0.0,  R_start_palm)
        palm_R_traj.Append(1.0, R_end_palm)
        
        n = N_list
        q0s = q_start
        q0e = q_end

        t_sol = []
        q_iiwa_sol = []
        q_hand_sol = []
        q_sol = []
        for i in range(N_list):
            t = float(i+1)/(N_list+1)

            if i%2 ==0: # From start
                t_now = (1)/(n+1)
                T_now = palm_T_traj.value(t_now)
                T_end = palm_T_traj.value(1)
                R_now = palm_R_traj.value(t_now)
                R_end = palm_R_traj.value(1)
                q0 = q0s*(1-t_now) +q0e*t_now

            else: # from end
                t_now = float((n)/(n+1))
                T_start = palm_T_traj.value(0)
                R_start = palm_R_traj.value(0)
                T_now = palm_T_traj.value(t_now)
                R_now = palm_R_traj.value(t_now)
                q0 = q0s*(1-t_now) +q0e*t_now

            T_offset = np.array([[0.001], [0.001], [0.001]])
            c = ik.AddPositionConstraint(
                self.palm_frame , [0, 0, 0], self.plant.world_frame(),
                T_now-T_offset, T_now+T_offset)


            c2=ik.AddOrientationConstraint(
                frameAbar = self.palm_frame, 
                R_AbarA = RotationMatrix(), 
                frameBbar = self.plant.world_frame(),
                R_BbarB = RotationMatrix(R_now), 
                theta_bound = 0.8)
            
            prog = ik.get_mutable_prog()
            q = ik.q()

            prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
            prog.SetInitialGuess(q, q0)
            result = Solve(ik.prog())

            q = self.plant.GetPositions(self.plant_context)
            
            q_iiwa = self.plant.GetPositions(self.plant_context, 
                        self.plant.GetModelInstanceByName("iiwa7"))
            q_hand = self.plant.GetPositions(self.plant_context, 
                        self.plant.GetModelInstanceByName("allegro_hand_right"))  

            X_palm = get_pose(self.palm_frame)
            T_palm = X_start_palm.translation()
            R_palm = X_start_palm.rotation()

            if i%2 ==0: # From start
                q0s  = q
                T_start = T_palm
                R_start = R_palm
            else:
                q0e  = q
                T_end = T_palm
                R_end = R_palm
            
            
            t_sol.append(t)
            q_iiwa_sol.append(q_iiwa)
            q_hand_sol.append(q_hand)
            q_sol.append(q)
            n-=1

            if n!=0:

                palm_T_traj = PiecewisePolynomial.FirstOrderHold(
                    [0.0, 1.0],
                    np.vstack([[np.squeeze(T_start_palm)],
                                [np.squeeze(T_end)]]).T)

                palm_R_traj = PiecewiseQuaternionSlerp()
                palm_R_traj.Append(0.0, RotationMatrix(R_start))
                palm_R_traj.Append(1.0, RotationMatrix(R_end))

        return  t_sol, q_iiwa_sol, q_hand_sol,q_sol

    def ik_press_chord(self, q0, note_on_fingers):

        ik = InverseKinematics(self.plant, self.plant_context)

        T_const_fingertip_flat_lower = np.array([-self.key_size[0]*0.005,-self.key_size[1]*0.3,-self.key_size[2]*0.05])
        T_const_fingertip_flat_upper = np.array([+self.key_size[0]*0.005,+self.key_size[1]*0.1,+self.key_size[2]*0.2])

        T_const_fingertip_sharp_lower = np.array([-self.key_size[0]*0.005,-self.key_size[1]*0.35,-self.key_size[2]*0.05])
        T_const_fingertip_sharp_upper = np.array([+self.key_size[0]*0.005,+self.key_size[1]*0.3,+self.key_size[2]*0.2])

        T_center = self.note_dictionary[self.key_list[18]].pos
        T_const_release_lower = np.array([-1,-1, self.key_size[2]*1.2])
        T_const_release_upper = np.array([1,1,self.key_size[2]*10])

        T_average= np.array([0.0,0.0,0.0])       
        N_playing_key =0

        for idx, note in enumerate(note_on_fingers):
            if note != None and "none" in note:
                T_key = T_center
                T_lower = T_key + T_const_release_lower
                T_upper = T_key + T_const_release_upper
            else:
                key_type = self.note_dictionary[note].keytype

                if key_type =="flat":
                    T_key = self.note_dictionary[note].pos
                    T_lower = T_key + T_const_fingertip_flat_lower
                    T_upper = T_key + T_const_fingertip_flat_upper
                else :
                    T_key = self.note_dictionary[note].pos
                    T_lower = T_key + T_const_fingertip_sharp_lower
                    T_upper = T_key + T_const_fingertip_sharp_upper
                    
                T_average +=T_key
                N_playing_key+=1

            c = ik.AddPositionConstraint(
                self.finger_frames[idx], [0, 0, 0],self.plant.world_frame(),
                T_lower, T_upper)

            if "none" in note:
                1 # Do Nothing

            else:
                if key_type == "flat":
                    c = ik.AddAngleBetweenVectorsConstraint(
                        self.finger_frames[idx], [0, 0, 1],
                        self.plant.world_frame(), [0,0,-1],
                        0, np.pi/2*0.3
                    )
                    c = ik.AddAngleBetweenVectorsConstraint(
                        self.finger_frames[idx], [0, 0, 1],
                        self.plant.world_frame(), [1,0,0],
                        np.pi/2*0.97, np.pi/2*1.03
                    )

                    c = ik.AddAngleBetweenVectorsConstraint(
                        self.finger_frames[idx], [0, 0, 1],
                        self.plant.world_frame(), [1,0,0],
                        np.pi/2*0.9, np.pi/2*1.3
                    )
                else: #"sharp"
                    c = ik.AddAngleBetweenVectorsConstraint(
                        self.finger_frames[idx], [0, 0, 1],
                        self.plant.world_frame(), [0,0,-1],
                        0, np.pi/2*0.3
                    )
                    c = ik.AddAngleBetweenVectorsConstraint(
                        self.finger_frames[idx], [0, 0, 1],
                        self.plant.world_frame(), [1,0,0],
                        np.pi/2*0.7, np.pi/2*1.3
                    )

        if N_playing_key >0:
            T_average = T_average/N_playing_key
        else:
            T_average = T_center

        T_const_palm_lower = np.array([-1,-self.key_size[1]*5,self.key_size[2]*1])
        T_const_palm_upper = np.array([+1,self.key_size[1]*5,self.key_size[2]*10])


        T_const_palm_rest_lower = np.array([-1,-self.key_size[1]*5,self.key_size[2]*1.5])
        T_const_palm_rest_upper = np.array([+1,0.5,self.key_size[2]*10])

        if N_playing_key>0:

            T_palm_lower =T_average + T_const_palm_lower
            T_palm_upper =T_average + T_const_palm_upper

            if N_playing_key==1:
                c = ik.AddAngleBetweenVectorsConstraint(
                    self.palm_frame, [0, 0, 1],
                    self.plant.world_frame(), [0,1,0],
                    0, np.pi/2*0.6
                )
            else:
                c = ik.AddAngleBetweenVectorsConstraint(
                    self.palm_frame, [0, 0, 1],
                    self.plant.world_frame(), [0,1,0],
                    0, np.pi/2*0.8
                )

            c = ik.AddAngleBetweenVectorsConstraint(
                self.palm_frame, [1, 0, 0],
                self.plant.world_frame(), [0,0,-1],
                0, np.pi/2*0.5
            )               
            # c = ik.AddAngleBetweenVectorsConstraint(
            #     self.palm_frame, [1, 0, 0],
            #     self.plant.world_frame(), [-1,0,0],
            #     0, np.pi/2*0.5
            # )               
        else:
            T_palm_lower =T_average + T_const_palm_rest_lower
            T_palm_upper =T_average + T_const_palm_rest_upper
            
            # c = ik.AddAngleBetweenVectorsConstraint(
            #     self.palm_frame, [0, 0, 1],
            #     self.plant.world_frame(), [0,1,0],
            #     0, np.pi/2*0.4
            # )         

            # c = ik.AddAngleBetweenVectorsConstraint(
            #     self.palm_frame, [1, 0, 0],
            #     self.plant.world_frame(), [0,0,-1],
            #     0, np.pi/2*0.5
            # )                      

        c = ik.AddPositionConstraint(
            self.palm_frame , [0, 0, 0], self.plant.world_frame(),
            T_palm_lower, T_palm_upper)

        # T_const_hand_lower = np.array([-2,-2,self.key_size[2]*0.2])
        # T_const_hand_upper = np.array([+2,2,2])
        # T_hand_lower = T_center + T_const_hand_lower
        # T_hand_upper = T_center + T_const_hand_upper

        # c = ik.AddPositionConstraint(
        #     self.hand_frame, [0, 0, 0],self.plant.world_frame(),
        #     T_hand_lower, T_palm_upper)
                
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = Solve(ik.prog())

        q_iiwa = self.plant.GetPositions(self.plant_context, 
                    self.plant.GetModelInstanceByName("iiwa7"))
        q_hand = self.plant.GetPositions(self.plant_context, 
                    self.plant.GetModelInstanceByName("allegro_hand_right"))  

        return result.is_success(), q_iiwa, q_hand
    
    
    def music_sequence_to_trajectory(self, music_seq):
        # @ music_seq : list of Music_Note class
        _iiwa = self.plant.GetModelInstanceByName("iiwa7")
        _hand = self.plant.GetModelInstanceByName("allegro_hand_right")

        q0 = self.plant.GetPositions(self.plant_context)
        q0_iiwa = self.plant.GetPositions(self.plant_context,_iiwa)
        q0_hand = self.plant.GetPositions(self.plant_context,_hand)
        # q0_all = q0_init

        result_test, q_iiwa_init, q_hand_init,_ = self.ik_init(q0)
        q0 = self.plant.GetPositions(self.plant_context)
        q0_init = q0
        t_init = 0.1

        t = t_init
        c_hand_release = np.array([0.001,0.001,0.001,0.001])

        # q_iiwa_traj = PiecewisePolynomial.FirstOrderHold([0,t],np.vstack([q_iiwa_init, q_iiwa_init]).T)
        # q_hand_traj = PiecewisePolynomial.FirstOrderHold([0,t],np.vstack([q_hand_init, q_hand_init]).T)
        # q_all_traj = PiecewisePolynomial.FirstOrderHold([0,t],np.vstack([q0_init, q0_init]).T)
        
        # c_hand_traj =  PiecewisePolynomial.FirstOrderHold([0,t],np.vstack([c_hand_release, c_hand_release]).T)

        # self.plant.SetPositions(self.plant_context, _iiwa, q_iiwa)
        # self.plant.SetPositions(self.plant_context, _hand, q_hand)

        chord_none = "none"
        chord_result = []
        
        def notes_to_c_hand(notes):
            c_hand = np.array([0,0,0,0])
            for idx, note in enumerate(notes):
                if note != "none":
                    c_hand[idx] = 1.00

            return c_hand
                

        q0_past = self.plant.GetPositions(self.plant_context)
        t_transition_past = 3
        init_flag = False
        for idx, music_note in enumerate(music_seq):
            # q0 = self.plant.GetPositions(self.plant_context)
            # result_test, q_iiwa_hold, q_hand_hold, notes = self.ik_chord_to_finger(chord = music_note.name, octave = music_note.key_scale,q0 = q0)
            # self.plant.SetPositions(self.plant_context, _iiwa, q_iiwa_hold)
            # self.plant.SetPositions(self.pclant_context, _hand, q_hand_hold)


            if music_note.name =="none":

                    t_all = music_note.t_push+ music_note.t_release + music_note.t_hold + t_transition_past
                    # t+=t_all
                    
                    t_transition_past = t_all+ music_note.t_transition
                    # t_transition_past = music_note.t_transition

            else:
                # result_press, q_iiwa_push, q_hand_push, q_iiwa_release, q_hand_release, notes = self.ik_chord_to_finger(chord = music_note.name, octave = music_note.key_scale,q0 = q0)
                
                result_press, q_iiwa_prepress, q_hand_prepress, q_iiwa_press, q_hand_press, q_iiwa_release, q_hand_release, notes =  self.ik_chord_to_finger(chord = music_note.name, octave = music_note.key_scale,q0 = q0)
                c_hand_push = notes_to_c_hand(notes)

                self.plant.SetPositions(self.plant_context, _iiwa, q_iiwa_press)
                self.plant.SetPositions(self.plant_context, _hand, q_hand_press)
                q0_press = self.plant.GetPositions(self.plant_context)

                self.plant.SetPositions(self.plant_context, _iiwa, q_iiwa_prepress)
                self.plant.SetPositions(self.plant_context, _hand, q_hand_prepress)
                q0_prepress = self.plant.GetPositions(self.plant_context)

                self.plant.SetPositions(self.plant_context, _iiwa, q_iiwa_release)
                self.plant.SetPositions(self.plant_context, _hand, q_hand_release)
                q0_release = self.plant.GetPositions(self.plant_context)
                chord_result.append([music_note.name, str(music_note.key_scale), result_press])
                
                # q0_transition = (q0_release*0.3+q0_past*0.7)
                # result_transition, q_iiwa_transition, q_hand_transition = self.ik_transition(q0_transition)

                # self.plant.SetPositions(self.plant_context, _iiwa, q_iiwa_transition)
                # self.plant.SetPositions(self.plant_context, _hand, q_hand_transition)
                # q0_transition = self.plant.GetPositions(self.plant_context)

                # q0_transition = q0_release

                if init_flag==False:

                    q_iiwa_traj = PiecewisePolynomial.FirstOrderHold([0,0.5],np.vstack([q_iiwa_release, q_iiwa_release]).T)
                    q_hand_traj = PiecewisePolynomial.FirstOrderHold([0,0.5],np.vstack([q_hand_release, q_hand_release]).T)
                    q_all_traj = PiecewisePolynomial.FirstOrderHold([0,0.5],np.vstack([q0_release, q0_release]).T)
                    c_hand_traj =  PiecewisePolynomial.FirstOrderHold([0,0.5],np.vstack([c_hand_release, c_hand_release]).T)
                    self.q0_init = q0_release

                    t += t_transition_past
                    q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_release)  
                    q_hand_traj.AppendFirstOrderSegment(t, q_hand_release)
                    q_all_traj.AppendFirstOrderSegment(t, q0_release)
                    c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)
                    init_flag =True
                else:


                    t_transition_list, q_arm_list, a_hand_list,q_sol_list = self.ik_transition_arm(q0_past, q0_prepress,2)
                    for idx, t_ratio in enumerate(t_transition_list):
                        t_now = t_transition_past*0.8*t_ratio
                        q_iiwa_now = q_arm_list[idx]
                        q_hand_now = a_hand_list[idx]
                        q_all_now = q_sol_list[idx]
                        
                        q_iiwa_traj.AppendFirstOrderSegment(t+t_now, q_iiwa_now)
                        q_hand_traj.AppendFirstOrderSegment(t+t_now, q_hand_now)
                        q_all_traj.AppendFirstOrderSegment(t+t_now, q_all_now)

 

                    t+=t_transition_past*0.8
                    q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_prepress)
                    q_hand_traj.AppendFirstOrderSegment(t, q_hand_prepress)
                    q_all_traj.AppendFirstOrderSegment(t, q0_prepress)
                    c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)

                    t+=t_transition_past*0.2
                    q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_prepress)
                    q_hand_traj.AppendFirstOrderSegment(t, q_hand_prepress)
                    q_all_traj.AppendFirstOrderSegment(t, q0_prepress)
                    c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)


                t += music_note.t_push
                q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_press)
                q_hand_traj.AppendFirstOrderSegment(t, q_hand_press)
                q_all_traj.AppendFirstOrderSegment(t, q0_press)
                c_hand_traj.AppendFirstOrderSegment(t,c_hand_push)

                t += music_note.t_hold
                q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_press)
                q_hand_traj.AppendFirstOrderSegment(t, q_hand_press)
                q_all_traj.AppendFirstOrderSegment(t, q0_press)
                c_hand_traj.AppendFirstOrderSegment(t,c_hand_push)

                t += music_note.t_release
                q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_release)  
                q_hand_traj.AppendFirstOrderSegment(t, q_hand_release)
                q_all_traj.AppendFirstOrderSegment(t, q0_release)
                c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)
                
                t_transition_past = music_note.t_transition
                q0_past = q0_release
                q0 = q0_release
        t+=1
        q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_release)  
        q_hand_traj.AppendFirstOrderSegment(t, q_hand_release)
        q_all_traj.AppendFirstOrderSegment(t, q0_release)
        c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)
                
        # q0_transition = (q0_release*0.5+q0_init*0.5)
        # result_transition, q_iiwa_transition, q_hand_transition = self.ik_transition(q0_transition)
        # self.plant.SetPositions(self.plant_context, _iiwa, q_iiwa_transition)
        # self.plant.SetPositions(self.plant_context, _hand, q_hand_transition)
        # q0_transition = self.plant.GetPositions(self.plant_context)

        # t+=1
        # q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_transition)   
        # q_hand_traj.AppendFirstOrderSegment(t, q_hand_transition)
        # q_all_traj.AppendFirstOrderSegment(t, q0_transition)
        # c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)
        t+=4
        q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_init)   
        q_hand_traj.AppendFirstOrderSegment(t, q_hand_init)  
        q_all_traj.AppendFirstOrderSegment(t, q0_init)
        c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)
        t+=1
        q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_init)   
        q_hand_traj.AppendFirstOrderSegment(t, q_hand_init)  
        q_all_traj.AppendFirstOrderSegment(t, q0_init)
        c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)

        # t+=0.5
        # q_iiwa_traj.AppendFirstOrderSegment(t, q_iiwa_init)   
        # q_hand_traj.AppendFirstOrderSegment(t, q_hand_init)
        # q_all_traj.AppendFirstOrderSegment(t, q0_init)    
        # c_hand_traj.AppendFirstOrderSegment(t,c_hand_release)

        w_iiwa_traj = q_iiwa_traj.MakeDerivative()
        w_hand_traj = q_hand_traj.MakeDerivative()


        return q_iiwa_traj, q_hand_traj, q_all_traj, w_iiwa_traj, w_hand_traj, c_hand_traj, chord_result, self.q0_init

    def inverse_kinematics_test(self):

        q0 = self.plant.GetPositions(self.plant_context)

        key = []
        for key_name in (self.key_list):
            key.append(self.note_dictionary[key_name])

        self.test_cnt = 0
        def my_callback(context, pose):

            q0 = self.plant.GetPositions(self.plant_context)
            self.test_cnt +=1
            if self.test_cnt >=9:
                self.test_cnt = 0

            key_test =[]
            tt = self.test_cnt%3
            ts = int((self.test_cnt -tt)/3)
            if self.test_cnt%3 ==0:
                key_test.append(key[0+ts*12])
                key_test.append(key[0+ts*12])
                key_test.append(key[2+ts*12])
                key_test.append(key[4+ts*12])

            elif self.test_cnt%3 ==1:
                key_test.append(key[0+ts*12])
                key_test.append(key[0+ts*12])
                key_test.append(key[4+ts*12])
                key_test.append(key[7+ts*12])

            else:
                key_test.append(key[0+ts*12])
                key_test.append(key[8+ts*12])
                key_test.append(key[9+ts*12])
                key_test.append(key[10+ts*12])

            ik = InverseKinematics(self.plant, self.plant_context)

            # ik.AddPositionConstraint(
            #     self.finger_frames[0], [0, 0, 0], self.plant.world_frame(),
            #     pose.translation(), pose.translation())
            T_const_fingertip_lower = np.array([-self.key_size[0]*0.1,-self.key_size[1]*0.35,0])
            T_const_fingertip_upper = np.array([+self.key_size[0]*0.1,+self.key_size[1]*0.25,0])

            X_KR1 = RigidTransform(R=RotationMatrix.MakeYRotation(np.pi))
            X_KR2 = RigidTransform(R=RotationMatrix.MakeZRotation(-np.pi/2))
            R_FContact = (X_KR1.multiply(X_KR2)).rotation()

            T_average= np.array([0.0,0.0,0.0])
            N_finger_idx = 0
            c_list = []
            cc_list = []
         
            for idx in range(1,4):
                T_key = key_test[idx].pos
                T_average +=T_key
 
                T_lower = T_key + T_const_fingertip_lower
                T_upper = T_key + T_const_fingertip_upper
                
                c = ik.AddPositionConstraint(
                    self.finger_frames[idx], [0, 0, 0],self.plant.world_frame(),
                    T_lower, T_upper)
                c_list.append(c)

                cc = str(idx)+"Pos1"
                cc_list.append(cc)
                # ik.AddAngleBetweenVectorsConstraint(
                #     self.finger_frames[idx], [0, 0, -1],
                #     self.plant.world_frame(), [1,0,0],
                #     np.pi/2*0.9, np.pi/2*1.1
                # )
                # ik.AddAngleBetweenVectorsConstraint(
                #     self.finger_frames[idx], [0, 0, -1],
                #     self.plant.world_frame(), [0,1,0],
                #     np.pi/2*0.8, np.pi/2*1.2
                # )
                c = ik.AddAngleBetweenVectorsConstraint(
                    self.finger_frames[idx], [0, 0, 1],
                    self.plant.world_frame(), [0,0,-1],
                    0, np.pi/2*0.3
                )

                cc = str(idx)+"angle1"
                cc_list.append(cc)

                c_list.append(c)
                c = ik.AddAngleBetweenVectorsConstraint(
                    self.finger_frames[idx], [1, 0, 1],
                    self.plant.world_frame(), [0,-1,0],
                    0, np.pi/2*0.4
                )
                cc = str(idx)+"angle2"
                cc_list.append(cc)

                c_list.append(c)
                # c = ik.AddAngleBetweenVectorsConstraint(
                #     self.finger_frames[idx], [0, 1, 1],
                #     self.plant.world_frame(), [-1,0,0],
                #     0, np.pi/2*0.25
                # )
                # cc = str(idx)+"angle3"
                # cc_list.append(cc)
                
                c_list.append(c)
                N_finger_idx +=1

            T_average = T_average/N_finger_idx
            T_const_palm_lower = np.array([-0.5,-self.key_size[1],self.key_size[2]*0.05])
            T_const_palm_upper = np.array([+0.5,self.key_size[1],self.key_size[2]*3])
            
            T_palm_lower =T_average + T_const_palm_lower
            T_palm_upper =T_average + T_const_palm_upper
            c = ik.AddPositionConstraint(
                self.palm_frame , [0, 0, 0], self.plant.world_frame(),
                T_palm_lower, T_palm_upper)

            c = ik.AddPositionConstraint(
                self.finger_frames[0], [0, 0, 0],self.plant.world_frame(),
                T_palm_lower, T_palm_upper)   

            c_list.append(c)
            cc = "palm_pos1"
            cc_list.append(cc)
            ik.AddAngleBetweenVectorsConstraint(
                self.palm_frame, [0, 0, 1],
                self.plant.world_frame(), [0,1,0],
                0, np.pi/2*0.5
            )

            c_list.append(c)
            cc = "palm_angle1"
            cc_list.append(cc)
  
            prog = ik.get_mutable_prog()
            q = ik.q()
            prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
            prog.SetInitialGuess(q, q0)
            result = Solve(ik.prog())
            #print(c_list)
            #print(cc_list)
            # print(result.GetSolution())
            q_iiwa = self.plant.GetPositions(self.plant_context, 
                        self.plant.GetModelInstanceByName("iiwa7"))
            print(q_iiwa)
            #print(result.GetInfeasibleConstraints(prog,0.01))
            if self.test_cnt==8:
                clear_output(wait=True)

            #print((self.test_cnt))
            if result.is_success():
                print("IK success_{}".format(self.test_cnt))
            else:
                print("IK failure_{}".format(self.test_cnt))

        sliders = MeshcatPoseSliders(meshcat)
        
        # sliders.SetPose(self.plant.EvalBodyPoseInWorld(
        #     self.plant_context, self.plant.GetBodyByName("link_7",self.model_hand)))

        # set the initial z lower, to make the interaction interesting.
        sliders.SetXyz([0.4, 0.2, 0.65])
        sliders.Run(self.visualizer, self.context, my_callback)
        meshcat.DeleteAddedControls()
        meshcat.Delete()

        return

    def ik_chord_to_finger(self, chord, octave,q0):

        notes_key = generate_note_from_chord(chord, octave)
        N_sol, fidx, cost = self.find_finger_correspondence(notes_key)
        cost_idx= np.argsort(cost[:,0])

        result = False

        q0 = self.plant.GetPositions(self.plant_context)

        for i in range(N_sol):
            notes = ["none","none","none","none"]
            n_idx = 0
            
            for ff in fidx[cost_idx[i]] :

                notes[int(ff)] = notes_key[n_idx]

                n_idx+=1
            
            result_prepress, q_iiwa_prepress, q_hand_prepress = self.ik_prepress_chord(q0, notes)
            q0_prepress  = self.plant.GetPositions(self.plant_context)

            result_press, q_iiwa_press, q_hand_press = self.ik_press_chord(q0_prepress, notes)
            q0_press  = self.plant.GetPositions(self.plant_context)

            result_release, q_iiwa_release, q_hand_release = self.ik_prepress_chord(q0_press, notes)

            result =result_prepress and result_press and result_release

            if result:
                break

        if result == False:
            notes = ["none","none","none","none"]
            i=0
            n_idx=0
            for ff in fidx[cost_idx[i]] :
                notes[int(ff)] = notes_key[n_idx]
                n_idx+=1


            result_prepress, q_iiwa_prepress, q_hand_prepress = self.ik_prepress_chord(q0, notes)
            q0_prepress  = self.plant.GetPositions(self.plant_context)

            result_press, q_iiwa_press, q_hand_press = self.ik_press_chord(q0_prepress, notes)
            q0_press  = self.plant.GetPositions(self.plant_context)

            result_release, q_iiwa_release, q_hand_release = self.ik_prepress_chord(q0_press, notes)

            result =result_prepress and result_press and result_release

        # return result_test, q_iiwa, q_hand, notes
        return result, q_iiwa_prepress, q_hand_prepress, q_iiwa_press, q_hand_press, q_iiwa_release, q_hand_release, notes
        
    def inverse_kinematics_test2(self):

        q0 = self.plant.GetPositions(self.plant_context)

        key = []
        for key_name in (self.key_list):
            key.append(self.note_dictionary[key_name])

        self.test_cnt = 0

        def my_callback(context, pose):

            q0 = self.plant.GetPositions(self.plant_context)
            N_chord = len(chord_list)
            self.test_cnt +=1
            if self.test_cnt >=N_chord*3:
                clear_output(wait=True)
                self.test_cnt = 0

            ii = self.test_cnt%N_chord
            ss = int(self.test_cnt/N_chord)+1
            # c_test = "CM"
            result_test,_,_, q_iiwa, q_hand, _, _,notes = self.ik_chord_to_finger(chord = chord_list[ii], octave = ss,q0 = q0)

            # if(self.test_cnt %3 ==0):
            #     q0 = self.plant.GetPositions(self.plant_context)
            #     result_test, q_iiwa, q_hand = self.ik_init(q0=q0)

            _iiwa = self.plant.GetModelInstanceByName("iiwa7")
            _hand = self.plant.GetModelInstanceByName("allegro_hand_right")
            self.plant.SetPositions(self.plant_context, _iiwa, q_iiwa)
            self.plant.SetPositions(self.plant_context, _hand, q_hand)

            
            if result_test:
                print("IK success_{}, ocatve:{}, chord:{}, notes:{}".format(self.test_cnt,ss,chord_list[ii],notes))
            else:
                print("IK failure_{}, ocatve:{}, chord:{}, notes:{}".format(self.test_cnt,ss,chord_list[ii],notes))


        sliders = MeshcatPoseSliders(meshcat)
        
        # sliders.SetPose(self.plant.EvalBodyPoseInWq0_initorld(
        #     self.plant_context, self.plant.GetBodyByName("link_7",self.model_hand)))

        # # set the initial z lower, to make the interaction interesting.
        # sliders.SetXyz([0.4, 0.2, 0.65])
        sliders.Run(self.visualizer, self.context, my_callback)
        meshcat.DeleteAddedControls()
        meshcat.Delete()

        return                        

### Test and generate the trajectory for Salut D'Amour offline

1. Initialize plant
2. Generate trajectory

Test routine is commented out. You can run by uncomment it.

In [5]:
pianoBot_trjGen = Piano_Project()
# pianoBot_trjGen.visualize_kinematic_tree()
# pianoBot_trjGen.inverse_kinematics_test2()
# pianoBot_trjGen.forward_kinematics_demo()
# pianoBot_trjGen.visualize_diagratm()

# little_star_chords = generate_little_star_chords_ver()
# little_star_notes = generate_little_star_note_ver()
saulte_de_amur_chords_ext = generate_salute_de_amur_lowchord_5actave()
saulte_de_amur_highnotes_ext = generate_salute_de_amur_highchord_5actave()

q_iiwa_trj1, q_hand_trj1, q_all_trj1, w_iiwa_trj1,w_hand_trj1, c_hand_trj1, trj_ik_result1, q0_init1 = pianoBot_trjGen.music_sequence_to_trajectory(saulte_de_amur_chords_ext)
# print(trj_ik_result1)

q_iiwa_trj2, q_hand_trj2, q_all_trj2, w_iiwa_trj2,w_hand_trj2, c_hand_trj2, trj_ik_result2, q0_init2 = pianoBot_trjGen.music_sequence_to_trajectory(saulte_de_amur_highnotes_ext)
# print(trj_ik_result2)



### Test static trajectory of the system

Initialize the project with the given trajectory and run the static trajectory simulation.

In [ ]:
pianoBot_staticTrj = Piano_Project(
    sim="static", q_all_traj = q_all_trj1,
    w_all_traj = q_all_trj1
)

pianoBot_staticTrj.run_simulation_demo(q0_init1,1)

pianoBot_staticTrj = Piano_Project(
    sim="static", q_all_traj = q_all_trj2,
    w_all_traj = q_all_trj2,
)

pianoBot_staticTrj.run_simulation_demo(q0_init2,1)

### Dynamic Simulation

Run dynamic simulation with given desired trajectory and save the log

In [18]:

def save_log(log,dir):
    t= log.sample_times()
    x = log.data()
    t_name = dir+"_t.npy"
    x_name = dir+"_data.npy"
    np.save(t_name,t)
    np.save(x_name,x)

fname1 = "log/log_salute_damur_chords"
fname2 = "log/log_salut_damur_note"

q0_init1p = q0_init1
pianoBot_dynamic1 = Piano_Project(
    sim="dynamic", q_iiwa_traj = q_iiwa_trj1, q_hand_traj =q_hand_trj1,
    w_iiwa_traj = w_iiwa_trj1,w_hand_traj = w_hand_trj1 , c_hand_traj=c_hand_trj1
)
log_salute_de_amur_chords = pianoBot_dynamic1.run_simulation_demo(q0_init1,t_duration = 45)


save_log(log_salute_de_amur_chords, fname1)    

pianoBot_dynamic2 = Piano_Project(
    sim="dynamic", q_iiwa_traj = q_iiwa_trj2, q_hand_traj =q_hand_trj2,
    w_iiwa_traj = w_iiwa_trj2, w_hand_traj = w_hand_trj2 , c_hand_traj=c_hand_trj2
)
log_salute_de_amur_highnotes = pianoBot_dynamic2.run_simulation_demo(q0_init2,t_duration = 45)

save_log(log_salute_de_amur_highnotes, fname2)    


### Construct Sound Output from the log

Replay the result and generate sound output.

You can also run the precomputed trajectory by uncomenting the file name below.

In [10]:
# Uncomment this line to run pre-computed trajectories
# fname1 = "log/trj_precomputed_chords"
# fname2 = "log/trj_precomputed_notes"

def reconstruct_log_to_trajectory(log):
    t = log.sample_times()
    x = log.data()  

    q0 = x[:83,0]
    q1 = x[:83,1]

    N = np.shape(t)[0]
    q_all_traj = PiecewisePolynomial.FirstOrderHold([t[0],t[1]],np.vstack([q0, q1]).T)
        
    N_downsample = 10

    for idx in range(N_downsample,N):
        if idx%N_downsample ==0:

            q = x[:59+12+12,idx]
            t_now = t[idx]
            q_all_traj.AppendFirstOrderSegment(t_now, q)


    dq_all_traj = q_all_traj.MakeDerivative()
    
    return q_all_traj, dq_all_traj

def reconstruct_logdata_to_trajectory(t,x):
    q0 = x[:83,0]
    q1 = x[:83,1]

    N = np.shape(t)[0]
    q_all_traj = PiecewisePolynomial.FirstOrderHold([t[0],t[1]],np.vstack([q0, q1]).T)
        
    N_downsample = 100

    for idx in range(N_downsample,N):
        if idx%N_downsample ==0:

            q = x[:59+12+12,idx]
            t_now = t[idx]
            q_all_traj.AppendFirstOrderSegment(t_now, q)



    dq_all_traj = q_all_traj.MakeDerivative()
    
    return q_all_traj, dq_all_traj

def load_log(fdir):
    log = np.load(fdir)
    return log

salute_de_amur_chord_t = load_log(fname1+"_t.npy")
salute_de_amur_chord_x = load_log(fname1+"_data.npy")
salute_de_amur_highnote_t = load_log(fname2+"_t.npy")
salute_de_amur_highnote_x = load_log(fname2+"_data.npy")


q_salute_de_amur_chord_trj,dq_salute_de_amur_chord_trj, = reconstruct_logdata_to_trajectory(salute_de_amur_chord_t, salute_de_amur_chord_x)
q_salute_de_amur_hignnote_trj, dq_salute_de_amur_hignnote_trj = reconstruct_logdata_to_trajectory(salute_de_amur_highnote_t, salute_de_amur_highnote_x)




In [11]:
pianoBot_soundGen1 = Piano_Project(
    sim="static", q_all_traj = q_salute_de_amur_chord_trj,
    w_all_traj = dq_salute_de_amur_chord_trj
)

#pianoBot_soundGen1.visualize_diagram()
pianoBot_soundGen1.run_simulation_demo(q0_init1,play_rate=1, t_duration =45)


pianoBot_soundGen2 = Piano_Project(
    sim="static", q_all_traj = q_salute_de_amur_hignnote_trj,
    w_all_traj = dq_salute_de_amur_hignnote_trj
)
#pianoBot_soundGen2.visualize_diagram()
pianoBot_soundGen2.run_simulation_demo(q0_init2,play_rate=1,t_duration=45)

idx : 20, name: G#_1, q: 0.004451847235546912, volume: 0.1782259230994151
idx : 23, name: B_1, q: 0.009048033669399914, volume: 0.13973126215815346
idx : 28, name: E_2, q: 0.00781782796581146, volume: 0.14926202516891393
idx : 20, name: G#_1, q: 0.03680684490831189, volume: 0.7623353433628016
idx : 23, name: B_1, q: 0.03416233557061452, volume: 0.7050164980876044
idx : 28, name: E_2, q: 0.016820781437213352, volume: 0.3523296894548275
idx : 28, name: E_2, t_on: 0.04300000000000003
idx : 23, name: B_1, t_on: 0.046000000000000034
idx : 20, name: G#_1, t_on: 0.048000000000000036
Estop idx : 20, name: G#_1, t_on: 0.048000000000000036, t_off: 0.03300000000000002
Estop idx : 23, name: B_1, t_on: 0.046000000000000034, t_off: 0.03400000000000002
Estop idx : 28, name: E_2, t_on: 0.04300000000000003, t_off: 0.035000000000000024
idx : 20, name: G#_1, q: 0.07510004107913329, volume: 0.36677983499460215
idx : 23, name: B_1, q: 0.06805349496066468, volume: 0.3447578716994996
idx : 26, name: D_2, q: 